In [1]:
# pd.set_option("display.max_columns", None)
# pd.reset_option('display.max_columns')
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')

Baseline Scenario
- Uncomment Code Below For Individual scenario runs (rather than full model MP8-Basic, MP9-Moderate, MP10-Advanced)

In [2]:
import os

# import from cmu-tare-model package
from config import PROJECT_ROOT

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

print(f"PROJECT_ROOT (from config.py): {PROJECT_ROOT}")

# Construct the absolute path to the .py file
relative_path = os.path.join("cmu_tare_model", "model_scenarios", "tare_baseline_v2_1.ipynb")
file_path = os.path.join(PROJECT_ROOT, relative_path)

# On Windows, to avoid any path-escape quirks, convert backslashes to forward slashes
file_path = file_path.replace("\\", "/")

print(f"Running file: {file_path}")

# %run magic command to run a .py file and import variables into the current IPython session
# # If your path has spaces, wrap it in quotes:
%run -i {file_path} # If your path has NO spaces, no quotes needed.

# iPthon magic command to run a .py file and import variables into the current IPython session
# Now run it, importing variables into your current IPython session
# from IPython import get_ipython
# get_ipython().run_line_magic('run', f'-i {file_path}')  # If your path has NO spaces, no quotes needed.

# # exec() function to run a .py file and import variables into the current IPython session
# with open(file_path) as f:
#     code = compile(f.read(), file_path, 'exec')
#     exec(code)

print("Baseline Scenario - Model Run Complete")

Project root directory: c:\users\14128\research\cmu-tare-model
PROJECT_ROOT (from config.py): c:\users\14128\research\cmu-tare-model
Running file: c:/users/14128/research/cmu-tare-model/cmu_tare_model/model_scenarios/tare_baseline_v2_1.ipynb

-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------
      
Project root directory: c:\users\14128\research\cmu-tare-model
Result outputs will be exported here: c:\users\14128\research\cmu-tare-model\cmu_tare_model\output_results


Retrieved data for filename: baseline_metadata_and_annual_r

## Dataframe for Electric Resistance Cooking (MP7)

In [3]:
print(f"""
====================================================================================================================================================================
We assume the use of Electric Resistance (MP7) rather than Induction (MP8).
Electric Resistance is significantly cheaper and only slightly less efficient than Induction.
====================================================================================================================================================================
""")

# Measure Package 7
menu_mp = 7
input_mp = 'upgrade07'

filename = "upgrade07_metadata_and_annual_results.csv"
relative_path = os.path.join("cmu_tare_model", "data", "euss_data", "resstock_amy2018_release_1.1", "state", filename)
file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

# Fix DtypeWarning error in columns 'in.neighbors' and 'in.geometry_stories_low_rise'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_mp7 = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id' (unique identifier)
print(f"DATAFRAME SIZE before applying any filters: {df_euss_am_mp7.shape}")


# Filter for occupied homes
occupancy_filter = df_euss_am_mp7['in.vacancy_status'] == 'Occupied'
df_euss_am_mp7 = df_euss_am_mp7.loc[occupancy_filter]
print(f"DATAFRAME SIZE after filtering for 'Occupied' homes: {df_euss_am_mp7.shape}")

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_mp7['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_mp7 = df_euss_am_mp7.loc[house_type_filter]
print(f"DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: {df_euss_am_mp7.shape}")

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'

# Filter down to state or city
else:
    print(f"You chose to filter for: {input_state}")
    state_filter = df_euss_am_mp7['in.state'].eq(input_state)
    df_euss_am_mp7 = df_euss_am_mp7.loc[state_filter]

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
    # Filter to a city within the selected state
    else:
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")
        city_filter = df_euss_am_mp7['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_mp7 = df_euss_am_mp7.loc[city_filter]

# Display the filtered dataframe
print(f"DATAFRAME SIZE after applying geographic filter: {df_euss_am_mp7.shape}")
print(df_euss_am_mp7)


We assume the use of Electric Resistance (MP7) rather than Induction (MP8).
Electric Resistance is significantly cheaper and only slightly less efficient than Induction.

Retrieved data for filename: upgrade07_metadata_and_annual_results.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\euss_data\resstock_amy2018_release_1.1\state\upgrade07_metadata_and_annual_results.csv


DATAFRAME SIZE before applying any filters: (548916, 346)
DATAFRAME SIZE after filtering for 'Occupied' homes: (482597, 346)
DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: (331531, 346)
You chose to filter for: PA
You chose to analyze all of state: PA
DATAFRAME SIZE after applying geographic filter: (15651, 346)
         upgrade      weight  applicability  in.sqft  \
bldg_id                                                
119            7  242.131013           True   2152.0   
122            7  242.131013           True   2176.0   
1

## Dataframe used for other end-uses (MP8)

In [4]:
# Print debugging information
print_debug = True

In [5]:
# Measure Package 8
menu_mp = 8
input_mp = 'upgrade08'
scenario_name = 'Basic-BAU'
cost_scenario = 'BAU Costs'
grid_scenario = 'Current Electricity Grid'

print(f"""
====================================================================================================================================================================
MODEL SCENARIO
====================================================================================================================================================================
Scenario {scenario_name}:
Basic Retrofit: Measure Package {menu_mp}
{cost_scenario}
{grid_scenario}
====================================================================================================================================================================
""")

filename = "upgrade08_metadata_and_annual_results.csv"
relative_path = os.path.join("cmu_tare_model", "data", "euss_data", "resstock_amy2018_release_1.1", "state", filename)
file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

# Fix DtypeWarning error in columns 'in.neighbors' and 'in.geometry_stories_low_rise'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_mp8 = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id' (unique identifier)
print(f"DATAFRAME SIZE before applying any filters: {df_euss_am_mp8.shape}")

# Filter for occupied homes
occupancy_filter = df_euss_am_mp8['in.vacancy_status'] == 'Occupied'
df_euss_am_mp8 = df_euss_am_mp8.loc[occupancy_filter]
print(f"DATAFRAME SIZE after filtering for 'Occupied' homes: {df_euss_am_mp8.shape}")

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_mp8['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_mp8 = df_euss_am_mp8.loc[house_type_filter]
print(f"DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: {df_euss_am_mp8.shape}")

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'

# Filter down to state or city
else:
    print(f"You chose to filter for: {input_state}")
    state_filter = df_euss_am_mp8['in.state'].eq(input_state)
    df_euss_am_mp8 = df_euss_am_mp8.loc[state_filter]

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
    # Filter to a city within the selected state
    else:
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")
        city_filter = df_euss_am_mp8['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_mp8 = df_euss_am_mp8.loc[city_filter]

# Display the filtered dataframe
# Display the filtered dataframe
print(f"DATAFRAME SIZE after applying geographic filter: {df_euss_am_mp8.shape}")
print(df_euss_am_mp8)


MODEL SCENARIO
Scenario Basic-BAU:
Basic Retrofit: Measure Package 8
BAU Costs
Current Electricity Grid

Retrieved data for filename: upgrade08_metadata_and_annual_results.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\euss_data\resstock_amy2018_release_1.1\state\upgrade08_metadata_and_annual_results.csv


DATAFRAME SIZE before applying any filters: (548916, 347)
DATAFRAME SIZE after filtering for 'Occupied' homes: (482597, 347)
DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: (331531, 347)
You chose to filter for: PA
You chose to analyze all of state: PA
DATAFRAME SIZE after applying geographic filter: (15651, 347)
         upgrade      weight  applicability  in.sqft  \
bldg_id                                                
119            8  242.131013           True   2152.0   
122            8  242.131013           True   2176.0   
150            8  242.131013           True   1690.0   
153        

# Project Future Energy Consumption

In [6]:
print(F"""
====================================================================================================================================================================
LOAD EUSS DATA FOR MEASURE PACKAGE {menu_mp} (MP{menu_mp})
====================================================================================================================================================================
You'll notice that the number of rows differs from df_euss_am_mp7 and df_euss_am_mp8.
      - df_euss_am_baseline_home has fewer rows (representative dwelling units) because a tech filter was applied. 
      - df_euss_am_mp8_home will have the same number of rows as df_euss_am_baseline_home after df_enduse_compare function is run.
      - df_enduse_compare function performs an inner merge on the two dataframes, keeping only the rows that are present in both dataframes.
====================================================================================================================================================================
df_euss_am_mp8_home will be created by running the df_enduse_compare function (contains post-retrofit consumption data for the entire home in 2024).
process_euss_data.py file contains the function definition.
      
def df_enduse_compare(df_mp: pd.DataFrame, 
                      input_mp: str, 
                      menu_mp: int, 
                      df_baseline: pd.DataFrame, 
                      df_cooking_range: pd.DataFrame
) -> pd.DataFrame:

""")

# df_enduse_compare(df_mp, category, df_baseline):
df_euss_am_mp8_home = df_enduse_compare(
    df_mp = df_euss_am_mp8,
    input_mp=input_mp,
    menu_mp=menu_mp,
    df_baseline = df_euss_am_baseline_home,
    df_cooking_range=df_euss_am_mp7,
    )



LOAD EUSS DATA FOR MEASURE PACKAGE 8 (MP8)
You'll notice that the number of rows differs from df_euss_am_mp7 and df_euss_am_mp8.
      - df_euss_am_baseline_home has fewer rows (representative dwelling units) because a tech filter was applied. 
      - df_euss_am_mp8_home will have the same number of rows as df_euss_am_baseline_home after df_enduse_compare function is run.
      - df_enduse_compare function performs an inner merge on the two dataframes, keeping only the rows that are present in both dataframes.
df_euss_am_mp8_home will be created by running the df_enduse_compare function (contains post-retrofit consumption data for the entire home in 2024).
process_euss_data.py file contains the function definition.
      
def df_enduse_compare(df_mp: pd.DataFrame, 
                      input_mp: str, 
                      menu_mp: int, 
                      df_baseline: pd.DataFrame, 
                      df_cooking_range: pd.DataFrame
) -> pd.DataFrame:



Applying combined vali

In [7]:
print(f"""
====================================================================================================================================================================
PROJECT FUTURE ENERGY CONSUMPTION
====================================================================================================================================================================
project_future_energy_consumption.py file was used to calculate/project the annual energy consumption 
for each home in the dataframe.
      
def project_future_consumption(
    df: pd.DataFrame, 
    menu_mp: int,
    base_year: int = 2024
) -> Tuple[pd.DataFrame, pd.DataFrame]:

""")
from cmu_tare_model.energy_consumption_and_metadata.project_future_energy_consumption import *

# Make copies to avoid modifying the original dataframes
df_euss_am_mp8_home = df_euss_am_mp8_home.copy()
df_mp8_consumption = df_euss_am_mp8_home.copy()

# Project Future Energy Consumption
df_euss_am_mp8_home, df_mp8_consumption = project_future_consumption(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp,
    base_year=2024
    )

print(f"""
====================================================================================================================================================================
DATAFRAME FOR MEASURE PACKAGE {menu_mp} (MP{menu_mp}):

{df_euss_am_mp8_home}
     
DATAFRAME (Consumption Cols): df_mp8_consumption
      
{df_mp8_consumption}

""")


PROJECT FUTURE ENERGY CONSUMPTION
project_future_energy_consumption.py file was used to calculate/project the annual energy consumption 
for each home in the dataframe.
      
def project_future_consumption(
    df: pd.DataFrame, 
    menu_mp: int,
    base_year: int = 2024
) -> Tuple[pd.DataFrame, pd.DataFrame]:



Determining valid homes for calculations:
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)
Measure package calculation for waterHeating:
  - 14999 homes have valid baseline data
  - 15551 homes will receive retrofits
  - 14999 homes have both valid data AND will receive retrofits
  - 652 homes excluded (values will be NaN)
Measure package calculation for clothesDrying:
  - 14743 homes have valid baseline data
  - 14743 homes will receive retrofits
  - 14743 homes have both valid data AND will re

In [8]:
if print_debug:
    from cmu_tare_model.utils.create_sample_df import *

    # def create_sample_df(
    #     df: pd.DataFrame,
    #     include_groups: Optional[List[str]] = None,
    #     categories: Optional[List[str]] = None,
    #     scenarios: Optional[List[str]] = None,
    #     metrics: Optional[List[str]] = None,
    #     mp_number: int = 8,
    #     regex_patterns: Optional[Union[str, List[str]]] = None  # New parameter
    # ) -> pd.DataFrame:

    # df_sample_climate

    # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_heating', 'valid_fuel_heating', 'valid_tech_heating', 'include_heating', 'heating_consumption']
    )
    print(df_sample_heating)

    # Create a sample dataframe for the waterHeating category
    df_sample_waterHeating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['waterHeating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'valid_tech_waterHeating', 'include_waterHeating', 'waterHeating_consumption']
    )
    print(df_sample_waterHeating)

    # Create a sample dataframe for the clothesDrying category
    df_sample_clothesDrying = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['clothesDrying'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_consumption']
    )
    print(df_sample_clothesDrying)

    # Create a sample dataframe for the cooking category
    df_sample_cooking = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_consumption']
    )
    print(df_sample_cooking)

        base_heating_fuel                         heating_type  \
bldg_id                                                          
119           Natural Gas             Natural Gas Fuel Furnace   
122              Fuel Oil                Fuel Oil Fuel Furnace   
150           Electricity                Electricity Baseboard   
153              Fuel Oil                Fuel Oil Fuel Furnace   
162              Fuel Oil                Fuel Oil Fuel Furnace   
...                   ...                                  ...   
549882        Natural Gas  Natural Gas Fuel Wall/Floor Furnace   
549915        Electricity         Electricity Electric Furnace   
549937        Natural Gas             Natural Gas Fuel Furnace   
549963        Natural Gas             Natural Gas Fuel Furnace   
549989           Fuel Oil                Fuel Oil Fuel Furnace   

         base_electricity_heating_consumption  \
bldg_id                                         
119                                      0.

# PUBLIC IMPACTS: CLIMATE AND HEALTH DAMAGES
## Scenarios: No IRA and IRA-Reference

In [9]:
print("""
====================================================================================================================================================================
PUBLIC IMPACTS: DAMAGES FROM CLIMATE AND HEALTH-RELATED EMISSIONS
====================================================================================================================================================================

""")
from cmu_tare_model.public_impact.calculate_lifetime_climate_impacts_sensitivity import *
from cmu_tare_model.public_impact.calculate_lifetime_health_impacts_sensitivity import *

# Make copies from scenario consumption to keep df smaller
print("\n", "Creating dataframe to store marginal damages calculations ...")
# Climate damages: No IRA and IRA-Reference
df_mp8_noIRA_damages_climate = df_mp8_consumption.copy()
df_mp8_IRA_damages_climate = df_mp8_consumption.copy()

# Health damages: No IRA and IRA-Reference
df_mp8_noIRA_damages_health = df_mp8_consumption.copy()
df_mp8_IRA_damages_health = df_mp8_consumption.copy()



PUBLIC IMPACTS: DAMAGES FROM CLIMATE AND HEALTH-RELATED EMISSIONS



 Creating dataframe to store marginal damages calculations ...


## Future Climate Damages: No IRA and IRA-Reference

In [10]:
print("""
========== SCENARIO: No Inflation Reduction Act ==========
""")
df_euss_am_mp8_home, df_mp8_noIRA_damages_climate = calculate_lifetime_climate_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='No Inflation Reduction Act', 
    df_baseline_damages=df_baseline_damages_climate,
    verbose=False  # Add this parameter
    )

df_euss_am_mp8_home, df_mp8_noIRA_damages_health = calculate_lifetime_health_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='No Inflation Reduction Act', 
    df_baseline_damages=df_baseline_damages_health,
    debug=False,
    verbose=False  # Add this parameter
    )


print("""
========== SCENARIO: Inflation Reduction Act (AEO2023 Reference Case) ==========
""")
df_euss_am_mp8_home, df_mp8_IRA_damages_climate = calculate_lifetime_climate_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='AEO2023 Reference Case', 
    df_baseline_damages=df_baseline_damages_climate,
    verbose=False  # Add this parameter
    )


df_euss_am_mp8_home, df_mp8_IRA_damages_health = calculate_lifetime_health_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='AEO2023 Reference Case', 
    df_baseline_damages=df_baseline_damages_health,
    debug=False,
    verbose=False  # Add this parameter
    )


print(f"""  
====================================================================================================================================================================
Post-Retrofit (MP8) Marginal Damages: WHOLE-HOME
Scenario: No Inflation Reduction Act and AEO2023 Reference Case
====================================================================================================================================================================
calculate_emissions_damages.py file contains the definition for the calculate_marginal_damages function.
Additional information on emissions and damage factor lookups can be found in the calculate_emissions_damages.py file as well. 
      
CLIMATE DAMAGES: No IRA and IRA-Reference
--------------------------------------------------------
Climate Damages (No IRA): df_mp8_noIRA_damages_climate
{df_mp8_noIRA_damages_climate}

Climate Damages (IRA): df_mp8_IRA_damages_climate
{df_mp8_IRA_damages_climate}

HEALTH DAMAGES: No IRA and IRA-Reference
--------------------------------------------------------
Health Damages (No IRA): df_mp8_noIRA_damages_health
{df_mp8_noIRA_damages_health}

Health Damages (IRA): df_mp8_IRA_damages_health
{df_mp8_IRA_damages_health}

SUMMARY DATAFRAME FOR MP8: df_euss_am_mp8_home
{df_euss_am_mp8_home}
====================================================================================================================================================================
""")


========== SCENARIO: No Inflation Reduction Act ==========

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              

Verifying masking for all calculated columns:

Verifying masking for all calculated columns:
-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              

Ve

In [11]:
if print_debug:
    from cmu_tare_model.utils.create_sample_df import *

    # def create_sample_df(
    #     df: pd.DataFrame,
    #     include_groups: Optional[List[str]] = None,
    #     categories: Optional[List[str]] = None,
    #     scenarios: Optional[List[str]] = None,
    #     metrics: Optional[List[str]] = None,
    #     mp_number: int = 8,
    #     regex_patterns: Optional[Union[str, List[str]]] = None  # New parameter
    # ) -> pd.DataFrame:

    # df_sample_climate

    # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_heating', 'valid_fuel_heating', 'include_heating', 'heating_lifetime_damages']
    )
    print(df_sample_heating)

    # Create a sample dataframe for the waterHeating category
    df_sample_waterHeating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['waterHeating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'include_waterHeating', 'waterHeating_lifetime_damages']
    )
    print(df_sample_waterHeating)

    # Create a sample dataframe for the clothesDrying category
    df_sample_clothesDrying = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['clothesDrying'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_lifetime_damages']
    )
    print(df_sample_clothesDrying)

    # Create a sample dataframe for the cooking category
    df_sample_cooking = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_damages']
    )
    print(df_sample_cooking)

        base_heating_fuel                         heating_type  \
bldg_id                                                          
119           Natural Gas             Natural Gas Fuel Furnace   
122              Fuel Oil                Fuel Oil Fuel Furnace   
150           Electricity                Electricity Baseboard   
153              Fuel Oil                Fuel Oil Fuel Furnace   
162              Fuel Oil                Fuel Oil Fuel Furnace   
...                   ...                                  ...   
549882        Natural Gas  Natural Gas Fuel Wall/Floor Furnace   
549915        Electricity         Electricity Electric Furnace   
549937        Natural Gas             Natural Gas Fuel Furnace   
549963        Natural Gas             Natural Gas Fuel Furnace   
549989           Fuel Oil                Fuel Oil Fuel Furnace   

         base_electricity_heating_consumption  \
bldg_id                                         
119                                      0.

In [12]:
if print_debug:
    from cmu_tare_model.utils.create_sample_df import *

    # def create_sample_df(
    #     df: pd.DataFrame,
    #     include_groups: Optional[List[str]] = None,
    #     categories: Optional[List[str]] = None,
    #     scenarios: Optional[List[str]] = None,
    #     metrics: Optional[List[str]] = None,
    #     mp_number: int = 8,
    #     regex_patterns: Optional[Union[str, List[str]]] = None  # New parameter
    # ) -> pd.DataFrame:

    # df_sample_climate

    print("""
========== SUMMARY DATAFRAME WITH LIFETIME DAMAGES ==========
""")

    # Create a sample dataframe for the heating category
    df_main_sample = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_damages']
    )
    print(f"""
df_main_sample dataframe is created using df_euss_am_mp8_home:
-----------------------------------------------------------------------------

{df_main_sample}

-----------------------------------------------------------------------------
""")


    print("""
========== CLIMATE IMPACTS WITH ANNUAL AND LIFETIME ==========
""")

    # Create a sample dataframe for the heating category
    df_detailed_climate_noIRA = create_sample_df(
        df=df_mp8_noIRA_damages_climate,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'cooking_damages_health']
    )
    print(f"""
df_detailed_climate_noIRA dataframe is created using df_mp8_noIRA_damages_climate:
-----------------------------------------------------------------------------
          
{df_detailed_climate_noIRA}

-----------------------------------------------------------------------------
""")

    # Create a sample dataframe for the heating category
    df_detailed_climate_IRA = create_sample_df(
        df=df_mp8_IRA_damages_climate,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'cooking_damages_health']
    )
    print(f"""
df_detailed_climate_IRA dataframe is created using df_mp8_IRA_damages_climate:
-----------------------------------------------------------------------------
          
{df_detailed_climate_IRA}

-----------------------------------------------------------------------------
""")

    print("""
========== HEALTH IMPACTS WITH ANNUAL AND LIFETIME ==========
""")
    # Create a sample dataframe for the heating category
    df_detailed_health_noIRA = create_sample_df(
        df=df_mp8_noIRA_damages_health,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'cooking_damages_health']
    )
    print(f"""
df_detailed_health_noIRA dataframe is created using df_mp8_noIRA_damages_health:
-----------------------------------------------------------------------------
          
{df_detailed_health_noIRA}

-----------------------------------------------------------------------------
""")

    # Create a sample dataframe for the heating category
    df_detailed_health_IRA = create_sample_df(
        df=df_mp8_IRA_damages_health,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'cooking_damages_health']
    )
    print(f"""
df_detailed_health_IRA dataframe is created using df_mp8_IRA_damages_health:
-----------------------------------------------------------------------------
          
{df_detailed_health_IRA}

-----------------------------------------------------------------------------
""")


========== SUMMARY DATAFRAME WITH LIFETIME DAMAGES ==========


df_main_sample dataframe is created using df_euss_am_mp8_home:
-----------------------------------------------------------------------------

        base_cooking_fuel  base_electricity_cooking_consumption  \
bldg_id                                                           
119           Natural Gas                                 32.53   
122           Electricity                                   NaN   
150           Electricity                                   NaN   
153           Electricity                                   NaN   
162           Electricity                                   NaN   
...                   ...                                   ...   
549882        Electricity                                   NaN   
549915        Electricity                                   NaN   
549937        Natural Gas                                 44.25   
549963        Electricity                               

## Future Annual Fuel Costs: No IRA and IRA-Reference

In [13]:
print("""  
====================================================================================================================================================================
PRIVATE IMPACTS: LIFETIME FUEL COSTS
====================================================================================================================================================================
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
====================================================================================================================================================================
""")
from cmu_tare_model.private_impact.calculate_lifetime_fuel_costs import *

print("\n", "Creating dataframe to store annual fuel cost calculations ...")
# Annual fuel costs: No IRA and IRA-Reference
df_mp8_fuelCosts_noIRA = df_mp8_consumption.copy()
df_mp8_fuelCosts_IRA = df_mp8_consumption.copy()

# # Make copies to prevent overwriting the original dataframe and compare the differences
# df_euss_am_mp8_home = df_euss_am_mp8_home.copy()
# df_mp8_fuelCosts = df_euss_am_mp8_home.copy()

  
PRIVATE IMPACTS: LIFETIME FUEL COSTS
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs


 Creating dataframe to store annual fuel cost calculations ...


In [14]:
print("""
========== SCENARIO: No Inflation Reduction Act ==========
""")
df_euss_am_mp8_home, df_mp8_noIRA_fuelCosts = calculate_lifetime_fuel_costs(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    df_baseline_costs=df_baseline_fuel_costs  # Add this line
    )

print("""
========== SCENARIO: Inflation Reduction Act (AEO2023 Reference Case) ==========
""")
df_euss_am_mp8_home, df_mp8_IRA_fuelCosts = calculate_lifetime_fuel_costs(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    df_baseline_costs=df_baseline_fuel_costs  # Add this line
    )


print(f"""  
====================================================================================================================================================================
Creating Dataframes for Lifetime Fuel Costs ...

FUEL COSTS (No IRA): df_mp8_noIRA_fuelCosts
{df_mp8_noIRA_fuelCosts}

FUEL COSTS (IRA): df_mp8_IRA_fuelCosts
{df_mp8_IRA_fuelCosts}

SUMMARY DATAFRAME FOR MP8: df_euss_am_mp8_home
{df_euss_am_mp8_home}

====================================================================================================================================================================
""")


========== SCENARIO: No Inflation Reduction Act ==========

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              
Calculating Fuel Costs from 2024 to 2039 for heating
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)
Calculating Fuel Costs from 2024 to 2036 for waterHeating
Measure package calculation for waterHeating:
  - 14999 homes have valid baseline data
  - 15551 homes will receive retrofits
  - 14999 homes have both valid data AND will recei

In [15]:
if print_debug:
    from cmu_tare_model.utils.create_sample_df import *

    # def create_sample_df(
    #     df: pd.DataFrame,
    #     include_groups: Optional[List[str]] = None,
    #     categories: Optional[List[str]] = None,
    #     scenarios: Optional[List[str]] = None,
    #     metrics: Optional[List[str]] = None,
    #     mp_number: int = 8,
    #     regex_patterns: Optional[Union[str, List[str]]] = None  # New parameter
    # ) -> pd.DataFrame:

    # df_sample_climate

    # Create a sample dataframe for the heating category
    df_main_sample = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment', 'costs'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_fuelCost']
    )
    print(f"""
df_main_sample dataframe is created using df_euss_am_mp8_home:
-----------------------------------------------------------------------------

{df_main_sample}

-----------------------------------------------------------------------------
""")


    # Create a sample dataframe for the heating category
    df_detailed_test = create_sample_df(
        df=df_mp8_IRA_fuelCosts,
        include_groups=['base_equipment', 'costs'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'_fuelCost']
    )
    print(f"""
df_detailed_test dataframe is created using df_mp8_IRA_fuelCosts:
-----------------------------------------------------------------------------
          
{df_detailed_test}

-----------------------------------------------------------------------------
""")


    # # Create a sample dataframe for the waterHeating category
    # df_sample_waterHeating = create_sample_df(
    #     df=df_euss_am_mp8_home,
    #     include_groups=['base_equipment', 'costs'],
    #     categories=['waterHeating'],
    #     scenarios=['preIRA', 'iraRef'],
    #     metrics=[],
    #     mp_number=menu_mp,
    #     regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'include_waterHeating', 'waterHeating_lifetime_fuelCost']
    # )
    # print(df_sample_waterHeating)

    # # Create a sample dataframe for the clothesDrying category
    # df_sample_clothesDrying = create_sample_df(
    #     df=df_euss_am_mp8_home,
    #     include_groups=['base_equipment', 'costs'],
    #     categories=['clothesDrying'],
    #     scenarios=['preIRA', 'iraRef'],
    #     metrics=[],
    #     mp_number=menu_mp,
    #     regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_lifetime_fuelCost']
    # )
    # print(df_sample_clothesDrying)

    # # Create a sample dataframe for the cooking category
    # df_sample_cooking = create_sample_df(
    #     df=df_euss_am_mp8_home,
    #     include_groups=['base_equipment', 'costs'],
    #     categories=['cooking'],
    #     scenarios=['preIRA', 'iraRef'],
    #     metrics=[],
    #     mp_number=menu_mp,
    #     regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_fuelCost']
    # )
    # print(df_sample_cooking)


df_main_sample dataframe is created using df_euss_am_mp8_home:
-----------------------------------------------------------------------------

        base_cooking_fuel  base_electricity_cooking_consumption  \
bldg_id                                                           
119           Natural Gas                                 32.53   
122           Electricity                                   NaN   
150           Electricity                                   NaN   
153           Electricity                                   NaN   
162           Electricity                                   NaN   
...                   ...                                   ...   
549882        Electricity                                   NaN   
549915        Electricity                                   NaN   
549937        Natural Gas                                 44.25   
549963        Electricity                                   NaN   
549989            Propane                            

# Calculate Capital Costs and Rebate Amounts

## Calculate Capital Costs (Applicable to All Scenarios)

In [16]:
print("""
====================================================================================================================================================================
PRIVATE IMPACTS: NET CAPITAL COSTS AND TOTAL CAPITAL COSTS
====================================================================================================================================================================
- Step 1: Calculate annual operating (fuel) costs ----- DONE -----
- Step 2: Calculate equipment capital costs (For space heating, include ductwork and weatherization (MP9 and MP10))
- Step 3: Calculate replacement cost (replacing existing piece of eqipment with similar technology)
- Step 4: Calculate net equipment capital costs
- Step 5: Calculate private NPV
====================================================================================================================================================================
""")


PRIVATE IMPACTS: NET CAPITAL COSTS AND TOTAL CAPITAL COSTS
- Step 1: Calculate annual operating (fuel) costs ----- DONE -----
- Step 2: Calculate equipment capital costs (For space heating, include ductwork and weatherization (MP9 and MP10))
- Step 3: Calculate replacement cost (replacing existing piece of eqipment with similar technology)
- Step 4: Calculate net equipment capital costs
- Step 5: Calculate private NPV



In [17]:
# UPDATED MARCH 24, 2025 @ 4:30 PM - REMOVED RSMEANS CCI ADJUSTMENTS
from cmu_tare_model.utils.inflation_adjustment import *

# Collect Capital Cost Data for different End-uses
filename = "tare_retrofit_costs_cpi.xlsx"
relative_path = os.path.join("cmu_tare_model", "data", "retrofit_costs", filename)
file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_heating_retrofit_costs = pd.read_excel(io=file_path, sheet_name='heating_costs')
df_waterHeating_retrofit_costs = pd.read_excel(io=file_path, sheet_name='waterHeating_costs')
df_clothesDrying_retrofit_costs = pd.read_excel(io=file_path, sheet_name='clothesDrying_costs')
df_cooking_retrofit_costs = pd.read_excel(io=file_path, sheet_name='cooking_costs')
df_enclosure_retrofit_costs = pd.read_excel(io=file_path, sheet_name='enclosure_upgrade_costs')

Retrieved data for filename: tare_retrofit_costs_cpi.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\retrofit_costs\tare_retrofit_costs_cpi.xlsx




### Space Heating and No Enclosure Upgrade

#### Space Heating Capital Costs

In [18]:
print("""
====================================================================================================================================================================
Capital Costs: Space Heating
====================================================================================================================================================================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...
""")
# UPDATED MARCH 24, 2025 @ 4:30 PM - REMOVED RSMEANS CCI ADJUSTMENTS
from cmu_tare_model.private_impact.calculations.calculate_equipment_installation_costs import *
from cmu_tare_model.private_impact.calculations.calculate_equipment_replacement_costs import *

# Columns to update
cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
    'cost_per_kBtuh_progressive', 'cost_per_kBtuh_reference', 'cost_per_kBtuh_conservative',
    'otherCost_progressive', 'otherCost_reference', 'otherCost_conservative'
]

# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_heating_retrofit_costs[column] = round((df_heating_retrofit_costs[column] * df_heating_retrofit_costs['cpi_ratio'] * df_heating_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_heating_equipment_cost = df_heating_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')

# Call the function and obtain equipment specifications
print("Obtaining system specs ...")
df_euss_am_mp8_home = obtain_heating_system_specs(df=df_euss_am_mp8_home)

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Heat Pump for Space Heating (No Enclosure Upgrade) ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_heating_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='heating')

# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_heating_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='heating')

# Call the function and calculate installation premium based on existing housing characteristics
# calculate_heating_installation_premium(df, menu_mp, cpi_ratio_2023_2013)
print("Calculating Space Heating Specific Premiums (Ex: Removing Hydronic Boiler) ...")
df_euss_am_mp8_home = calculate_heating_installation_premium(df=df_euss_am_mp8_home,
                                                             menu_mp=menu_mp,
                                                             cpi_ratio_2023_2013=cpi_ratio_2023_2013)

# Display the df
print(df_euss_am_mp8_home)


Capital Costs: Space Heating

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...

Obtaining system specs ...
Calculating Cost of Retrofit Upgrade: Heat Pump for Space Heating (No Enclosure Upgrade) ...
Starting heating installation cost calculation with validation framework
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)
Found 12266 valid homes out of 15651 for heating installation
After tech filtering: 12266 homes remain valid for heating installation

Verifying masking for all calculated columns:
Masking 1 columns for category 'heating'
Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...
Starting heating replacement cost calculation with validation framework
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15649 homes will re

In [19]:
# # upgrade_hvac_heating_efficiency
# # baseline_AFUE  baseline_SEER  baseline_HSPF ugrade_newInstall_HSPF
# # Call the function and obtain equipment specifications
# print("Obtaining system specs ...")
# df_euss_am_mp8_home = obtain_heating_system_specs(df=df_euss_am_mp8_home)

if print_debug:
    # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['valid_fuel_heating', 'include_heating', 'baseline_AFUE', 'baseline_SEER', 'baseline_HSPF', 
                        'hvac_heating_efficiency', 'upgrade_hvac_', 'upgrade_heating','ugrade_newInstall_HSPF']
    )
    print(df_sample_heating)


        base_heating_fuel                         heating_type  \
bldg_id                                                          
119           Natural Gas             Natural Gas Fuel Furnace   
122              Fuel Oil                Fuel Oil Fuel Furnace   
150           Electricity                Electricity Baseboard   
153              Fuel Oil                Fuel Oil Fuel Furnace   
162              Fuel Oil                Fuel Oil Fuel Furnace   
...                   ...                                  ...   
549882        Natural Gas  Natural Gas Fuel Wall/Floor Furnace   
549915        Electricity         Electricity Electric Furnace   
549937        Natural Gas             Natural Gas Fuel Furnace   
549963        Natural Gas             Natural Gas Fuel Furnace   
549989           Fuel Oil                Fuel Oil Fuel Furnace   

         base_electricity_heating_consumption  \
bldg_id                                         
119                                      0.

### Water Heating

In [20]:
print("""
====================================================================================================================================================================
Capital Costs: Water Heating
====================================================================================================================================================================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...
""")

cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
    'cost_per_gallon_progressive', 'cost_per_gallon_reference', 'cost_per_gallon_conservative',
]

# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_waterHeating_retrofit_costs[column] = round((df_waterHeating_retrofit_costs[column] * df_waterHeating_retrofit_costs['cpi_ratio'] * df_waterHeating_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_waterHeating_equipment_cost = df_waterHeating_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')
# dict_waterHeating_equipment_cost

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Electric Heat Pump Water Heater ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_waterHeating_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='waterHeating')

# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_waterHeating_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='waterHeating')

# Display the df
print(df_euss_am_mp8_home)


Capital Costs: Water Heating

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...

Calculating Cost of Retrofit Upgrade: Electric Heat Pump Water Heater ...
Starting waterHeating installation cost calculation with validation framework
Measure package calculation for waterHeating:
  - 14999 homes have valid baseline data
  - 15551 homes will receive retrofits
  - 14999 homes have both valid data AND will receive retrofits
  - 652 homes excluded (values will be NaN)
Found 14999 valid homes out of 15651 for waterHeating installation
After tech filtering: 14999 homes remain valid for waterHeating installation

Verifying masking for all calculated columns:
Masking 1 columns for category 'waterHeating'
Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...
Starting waterHeating replacement cost calculation with validation framework
Measure package calculation for waterHeating:
  - 14999 homes have valid baseline data
  - 15551 homes will receive retrof

### Clothes Drying

In [21]:
print("""
====================================================================================================================================================================
Capital Costs: Clothes Drying
====================================================================================================================================================================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ... 
""")

# Columns to update
cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
]
 
# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_clothesDrying_retrofit_costs[column] = round((df_clothesDrying_retrofit_costs[column] * df_clothesDrying_retrofit_costs['cpi_ratio'] * df_clothesDrying_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_clothesDrying_equipment_cost = df_clothesDrying_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')
# dict_clothesDrying_equipment_cost

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Ventless Heat Pump Clothes Dryer ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_clothesDrying_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='clothesDrying')


# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_clothesDrying_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='clothesDrying')

# Display the df
print(df_euss_am_mp8_home)


Capital Costs: Clothes Drying

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ... 

Calculating Cost of Retrofit Upgrade: Ventless Heat Pump Clothes Dryer ...
Starting clothesDrying installation cost calculation with validation framework
Measure package calculation for clothesDrying:
  - 14743 homes have valid baseline data
  - 14743 homes will receive retrofits
  - 14743 homes have both valid data AND will receive retrofits
  - 908 homes excluded (values will be NaN)
Found 14743 valid homes out of 15651 for clothesDrying installation
After tech filtering: 14743 homes remain valid for clothesDrying installation

Verifying masking for all calculated columns:
Masking 1 columns for category 'clothesDrying'
Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...
Starting clothesDrying replacement cost calculation with validation framework
Measure package calculation for clothesDrying:
  - 14743 homes have valid baseline data
  - 14743 homes will rece

### Cooking

In [22]:
print("""
====================================================================================================================================================================
Capital Costs: Cooking
====================================================================================================================================================================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...      
""")

# Columns to update
cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
]
 
# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_cooking_retrofit_costs[column] = round((df_cooking_retrofit_costs[column] * df_cooking_retrofit_costs['cpi_ratio'] * df_cooking_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_cooking_equipment_cost = df_cooking_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')
# dict_cooking_equipment_cost

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Electric Resistance Range ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_cooking_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='cooking')

# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_cooking_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='cooking')

# Display the df
print(df_euss_am_mp8_home)


Capital Costs: Cooking

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...      

Calculating Cost of Retrofit Upgrade: Electric Resistance Range ...
Starting cooking installation cost calculation with validation framework
Measure package calculation for cooking:
  - 7867 homes have valid baseline data
  - 7867 homes will receive retrofits
  - 7867 homes have both valid data AND will receive retrofits
  - 7784 homes excluded (values will be NaN)
Found 7867 valid homes out of 15651 for cooking installation
After tech filtering: 7867 homes remain valid for cooking installation

Verifying masking for all calculated columns:
Masking 1 columns for category 'cooking'
Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...
Starting cooking replacement cost calculation with validation framework
Measure package calculation for cooking:
  - 7867 homes have valid baseline data
  - 7867 homes will receive retrofits
  - 7867 homes have both valid data AND wil

 ## Calculate Rebate Amounts (Applicable to IRA-Reference)

In [23]:
from cmu_tare_model.private_impact.data_processing.determine_rebate_eligibility_and_amount import *

# Determine Percent AMI and Rebate Amounts
# This needs to be done before running the calculate_percent_AMI function
df_euss_am_mp8_home = df_euss_am_mp8_home.copy()

# calculate_percent_AMI(df_results_IRA, df_county_medianIncome):
df_euss_am_mp8_home = calculate_percent_AMI(df_results_IRA=df_euss_am_mp8_home)

print("Calculating rebate amounts for Space Heating ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="heating",
                                          menu_mp=menu_mp)

print("Calculating rebate amounts for Water Heating ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="waterHeating",
                                          menu_mp=menu_mp)

print("Calculating rebate amounts for Clothes Drying ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="clothesDrying",
                                          menu_mp=menu_mp)

print("Calculating rebate amounts for Cooking ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="cooking",
                                          menu_mp=menu_mp)

print(f"""
====================================================================================================================================================================
CALCULATE HOUSEHOLD PERCENT AREA MEDIAN INCOME (%AMI) AND REBATE ELIGIBILITY/AMOUNTS
====================================================================================================================================================================
determine_rebate_eligibility_and_amount.py file contains the function definitions for calculating rebate amounts and determining household %AMI.
process_income_data_for_rebates.py file contains additional information on data sources and procedures used to process data for determine_rebate_eligibility_and_amount.py file.

DATAFRAME: df_euss_am_mp8_home AFTER CALCULATING REBATE AMOUNTS
{df_euss_am_mp8_home}

====================================================================================================================================================================
""")

Calculating rebate amounts for Space Heating ...
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)

Verifying masking for all calculated columns:
Calculating rebate amounts for Water Heating ...
Measure package calculation for waterHeating:
  - 14999 homes have valid baseline data
  - 15551 homes will receive retrofits
  - 14999 homes have both valid data AND will receive retrofits
  - 652 homes excluded (values will be NaN)

Verifying masking for all calculated columns:
Calculating rebate amounts for Clothes Drying ...
Measure package calculation for clothesDrying:
  - 14743 homes have valid baseline data
  - 14743 homes will receive retrofits
  - 14743 homes have both valid data AND will receive retrofits
  - 908 homes excluded (values will be NaN)

Verifying masking for all calculated columns:
Calculating 

# SCENARIO ANALYSIS: Basic Pre-IRA Scenario
## - NREL End-Use Savings Shapes Database: Measure Package 8
## - AEO2023 No Inflation Reduction Act
## - Cambium 2021 MidCase Scenario

In [24]:
# Measure Package 8
scenario_name = 'No Inflation Reduction Act'
cost_scenario = 'Fuel Costs: AEO2023 No Inflation Reduction Act'
grid_scenario = 'Electricity Grid: Cambium 2021 MidCase Scenario'

print(f"""
====================================================================================================================================================================
MODEL SCENARIO
====================================================================================================================================================================
Scenario {scenario_name}:
Basic Retrofit: Measure Package {menu_mp}
{cost_scenario}
{grid_scenario}
====================================================================================================================================================================
""")


MODEL SCENARIO
Scenario No Inflation Reduction Act:
Basic Retrofit: Measure Package 8
Fuel Costs: AEO2023 No Inflation Reduction Act
Electricity Grid: Cambium 2021 MidCase Scenario



In [25]:
from cmu_tare_model.public_impact.calculate_lifetime_public_impact_sensitivity import *

# # LAST UPDATED APRIL 13, 2025 @ 12:30PM
# def calculate_public_npv(
#     df: pd.DataFrame, 
#     df_baseline_climate: pd.DataFrame, 
#     df_baseline_health: pd.DataFrame,
#     df_mp_climate: pd.DataFrame,
#     df_mp_health: pd.DataFrame,
#     menu_mp: str, 
#     policy_scenario: str, 
#     rcm_model: str,
#     base_year: int = 2024,
#     discounting_method: str = 'public',
# ) -> pd.DataFrame:

# Create copies to prevent overwriting the original dataframe and compare the differences
df_euss_am_mp8_home_ap2 = df_euss_am_mp8_home.copy()
df_euss_am_mp8_home_easiur = df_euss_am_mp8_home.copy()
df_euss_am_mp8_home_inmap = df_euss_am_mp8_home.copy()

# Example of how to call the updated calculate_public_npv function

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = calculate_public_npv(
    df=df_euss_am_mp8_home_ap2,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_noIRA_damages_climate,
    df_mp_health=df_mp8_noIRA_damages_health,
    menu_mp="8",
    policy_scenario='No Inflation Reduction Act',
    rcm_model='AP2',
    base_year=2024,
    discounting_method='public'
)

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = calculate_public_npv(
    df=df_euss_am_mp8_home_easiur,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_noIRA_damages_climate,
    df_mp_health=df_mp8_noIRA_damages_health,
    menu_mp="8",
    policy_scenario='No Inflation Reduction Act',
    rcm_model='EASIUR',
    base_year=2024,
    discounting_method='public'
)

# ============= INMAP =============
df_euss_am_mp8_home_inmap = calculate_public_npv(
    df=df_euss_am_mp8_home_inmap,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_noIRA_damages_climate,
    df_mp_health=df_mp8_noIRA_damages_health,
    menu_mp="8",
    policy_scenario='No Inflation Reduction Act',
    rcm_model='InMAP',
    base_year=2024,
    discounting_method='public'
)

print(f"""  
====================================================================================================================================================================
PUBLIC PERSPECTIVE COSTS AND BENEFITS: NO INFLATION REDUCTION ACT
====================================================================================================================================================================
calculate_lifetime_public_impact.py file contains the definition for the calculate_public_npv function.
Additional information on emissions/damage factor lookups as well as marginal damages calculation methods can be found in the calculate_emissions_damages.py file. 
      
DATAFRAME FOR MP8 AFTER CALCULATING PUBLIC NPV: df_euss_am_mp8_home

AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              
Baseline Climate DataFrame: ✓ Valid
Baseline Health DataFrame: ✓ Valid
Retrofit Climate DataFrame: ✓ Valid
Retrofit Health DataFrame: ✓ Valid
-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              
-- Scenario: No 

In [26]:
# Check all columns with "public_npv" in their name
npv_columns = [col for col in df_euss_am_mp8_home_ap2.columns if 'public_npv' in col]
print(f"Found {len(npv_columns)} NPV columns")

Found 24 NPV columns


In [27]:
df_euss_am_mp8_home_ap2

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,preIRA_mp8_clothesDrying_public_npv_central_AP2_h6c,preIRA_mp8_clothesDrying_climate_npv_upper,preIRA_mp8_clothesDrying_public_npv_upper_AP2_h6c,preIRA_mp8_cooking_climate_npv_lower,preIRA_mp8_cooking_health_npv_AP2_h6c,preIRA_mp8_cooking_public_npv_lower_AP2_h6c,preIRA_mp8_cooking_climate_npv_central,preIRA_mp8_cooking_public_npv_central_AP2_h6c,preIRA_mp8_cooking_climate_npv_upper,preIRA_mp8_cooking_public_npv_upper_AP2_h6c
bldg_id,,,,,,,,,,,,,,,,,,,,,
119,2152.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,-117.74,121.30,-30.69,7.27,-336.58,-329.31,21.95,-314.63,75.80,-260.77
122,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,In another census Place,G4200450,...,159.93,179.26,289.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,1690.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,In another census Place,G4201190,...,132.37,208.20,282.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,2663.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,574.56,643.99,1038.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549882,1202.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Not in a census Place,G4200110,...,247.75,288.69,455.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549915,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Lancaster,G4200710,...,415.47,502.81,777.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549937,885.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,115,RFCWc,PA,In another census Place,G4200050,...,-57.70,307.95,163.60,4.84,-152.22,-147.39,14.85,-137.37,51.41,-100.82


In [28]:
from cmu_tare_model.private_impact.calculate_lifetime_private_impact import *

# def calculate_private_NPV(
#         df: pd.DataFrame,
#         df_fuel_costs: pd.DataFrame,
#         df_baseline_costs: pd.DataFrame,
#         input_mp: str,
#         menu_mp: int,
#         policy_scenario: str,
#         discounting_method: str = 'private_fixed',
#         base_year: int = 2024,
#         verbose: bool = True
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = calculate_private_NPV(
    df=df_euss_am_mp8_home_ap2,
    df_fuel_costs=df_mp8_noIRA_fuelCosts,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='No Inflation Reduction Act',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = calculate_private_NPV(
    df=df_euss_am_mp8_home_easiur,
    df_fuel_costs=df_mp8_noIRA_fuelCosts,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='No Inflation Reduction Act',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )

# ============ InMAP =============
df_euss_am_mp8_home_inmap = calculate_private_NPV(
    df=df_euss_am_mp8_home_inmap,
    df_fuel_costs=df_mp8_noIRA_fuelCosts,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='No Inflation Reduction Act',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )


print(f"""  
====================================================================================================================================================================
PRIVATE PERSPECTIVE COSTS AND BENEFITS: NO INFLATION REDUCTION ACT
====================================================================================================================================================================
calculate_lifetime_private_impact.py file contains the definition for the calculate_private_NPV function.
Additional information on fuel price lookups can be found in the calculate_fuel_costs.py file. 

def calculate_private_NPV(
        df: pd.DataFrame,
        df_fuel_costs: pd.DataFrame,
        df_baseline_costs: pd.DataFrame,
        input_mp: str,
        menu_mp: int,
        policy_scenario: str,
        discounting_method: str = 'private_fixed',
        base_year: int = 2024,
        verbose: bool = True
) -> pd.DataFrame:

DATAFRAME FOR MP8 AFTER CALCULATING PRIVATE NPV: df_euss_am_mp8_home

AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              

Determining lifetime private impacts for category: heating with lifetime: 15
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)

Calculating costs for heating...
          input_mp: upgrade08, menu_mp: 8, policy_scenario: No Inflation Reduction Act
Calculating Private NPV for heating with lifetime: 15 years
          policy_scenario: No Inflation Reduction Act --> scenario_prefix: preIRA_mp8_
   

In [29]:
from cmu_tare_model.adoption_potential.determine_adoption_potential_sensitivity import *

# def adoption_decision(df: pd.DataFrame,
#                       menu_mp: int,
#                       policy_scenario: str,
#                       rcm_model: str,
#                       cr_function: str,
#                       climate_sensitivity: bool = False  # Default is false because we use $190USD2020/mt in Joseph et al. (2025)
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='AP2',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='AP2',
    cr_function='h6c',
    climate_sensitivity=False
    )

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='EASIUR',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='EASIUR',
    cr_function='h6c',
    climate_sensitivity=False
    )

# ============ InMAP =============
df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='InMAP',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='InMAP',
    cr_function='h6c',
    climate_sensitivity=False
    )


print(f"""
====================================================================================================================================================================
ADOPTION FEASIBILITY OF VARIOUS RETROFITS: NO INFLATION REDUCTION ACT
====================================================================================================================================================================
determine_adoption_potential.py file contains the definition for the adoption_decision function.

DATAFRAME FOR MP8 AFTER DETERMINING ADOPTION FEASIBILITY: df_euss_am_mp8_home
      
AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              

Calculating Adoption Potential for heating under 'No Inflation Reduction Act' Scenario...
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)
Found 12266 valid homes out of 15651 for heating adoption potential

 --- Public NPV Sensitivity ---
                              Climate Impact Sensitivity:
                                SCC Assumption (Bound): upper
                
                    

In [30]:
# print(f"Adoption potential for AP2: {df_euss_am_mp8_home_ap2.columns.to_list()}")
# print(f"Adoption potential for EASIUR: {df_euss_am_mp8_home_easiur.columns.to_list()}")
# print(f"Adoption potential for InMAP: {df_euss_am_mp8_home_inmap.columns.to_list()}")


# Basic IRA-Reference Scenario:
## - NREL End-Use Savings Shapes Database: Measure Package 8
## - AEO2023 REFERENCE CASE - HDD and Fuel Price Projections
## - Cambium 2023 MidCase Scenario

In [31]:
# Measure Package 8
scenario_name = 'Basic IRA-Reference'
cost_scenario = 'Fuel Costs: AEO2023 Reference Case'
grid_scenario = 'Electricity Grid: Cambium 2023 MidCase Scenario'

print(f"""
====================================================================================================================================================================
MODEL SCENARIO
====================================================================================================================================================================
Scenario {scenario_name}:
Basic Retrofit: Measure Package {menu_mp}
{cost_scenario}
{grid_scenario}
====================================================================================================================================================================
""")


MODEL SCENARIO
Scenario Basic IRA-Reference:
Basic Retrofit: Measure Package 8
Fuel Costs: AEO2023 Reference Case
Electricity Grid: Cambium 2023 MidCase Scenario



In [32]:
# from cmu_tare_model.public_impact.calculate_lifetime_public_impact_sensitivity import *

# # LAST UPDATED APRIL 13, 2025 @ 12:30PM
# def calculate_public_npv(
#     df: pd.DataFrame, 
#     df_baseline_climate: pd.DataFrame, 
#     df_baseline_health: pd.DataFrame,
#     df_mp_climate: pd.DataFrame,
#     df_mp_health: pd.DataFrame,
#     menu_mp: str, 
#     policy_scenario: str, 
#     rcm_model: str,
#     base_year: int = 2024,
#     discounting_method: str = 'public',
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = calculate_public_npv(
    df=df_euss_am_mp8_home_ap2,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_IRA_damages_climate,
    df_mp_health=df_mp8_IRA_damages_health,
    menu_mp="8",
    policy_scenario='AEO2023 Reference Case',
    rcm_model='AP2',
    base_year=2024,
    discounting_method='public'
)

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = calculate_public_npv(
    df=df_euss_am_mp8_home_easiur,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_IRA_damages_climate,
    df_mp_health=df_mp8_IRA_damages_health,
    menu_mp="8",
    policy_scenario='AEO2023 Reference Case',
    rcm_model='EASIUR',
    base_year=2024,
    discounting_method='public'
)

# ============= INMAP =============
df_euss_am_mp8_home_inmap = calculate_public_npv(
    df=df_euss_am_mp8_home_inmap,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_IRA_damages_climate,
    df_mp_health=df_mp8_IRA_damages_health,
    menu_mp="8",
    policy_scenario='AEO2023 Reference Case',
    rcm_model='InMAP',
    base_year=2024,
    discounting_method='public'
)

print(f"""  
====================================================================================================================================================================
PUBLIC PERSPECTIVE COSTS AND BENEFITS: AEO2023 Reference Case
====================================================================================================================================================================
calculate_lifetime_public_impact.py file contains the definition for the calculate_public_npv function.
Additional information on emissions/damage factor lookups as well as marginal damages calculation methods can be found in the calculate_emissions_damages.py file. 
      
DATAFRAME FOR MP8 AFTER CALCULATING PUBLIC NPV: df_euss_am_mp8_home

AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

-- Scenario: Inflation Reduction Act (IRA) Reference -- 
              scenario_prefix: 'iraRef_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_IRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_iraRef'
              
Baseline Climate DataFrame: ✓ Valid
Baseline Health DataFrame: ✓ Valid
Retrofit Climate DataFrame: ✓ Valid
Retrofit Health DataFrame: ✓ Valid
-- Scenario: Inflation Reduction Act (IRA) Reference -- 
              scenario_prefix: 'iraRef_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_IRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_iraRef'
             

In [33]:
# from cmu_tare_model.private_impact.calculate_lifetime_private_impact import *

# def calculate_private_NPV(
#         df: pd.DataFrame,
#         df_fuel_costs: pd.DataFrame,
#         df_baseline_costs: pd.DataFrame,
#         input_mp: str,
#         menu_mp: int,
#         policy_scenario: str,
#         discounting_method: str = 'private_fixed',
#         base_year: int = 2024,
#         verbose: bool = True
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = calculate_private_NPV(
    df=df_euss_am_mp8_home_ap2,
    df_fuel_costs=df_mp8_IRA_fuelCosts,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='AEO2023 Reference Case',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = calculate_private_NPV(
    df=df_euss_am_mp8_home_easiur,
    df_fuel_costs=df_mp8_IRA_fuelCosts,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='AEO2023 Reference Case',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )

# ============ InMAP =============
df_euss_am_mp8_home_inmap = calculate_private_NPV(
    df=df_euss_am_mp8_home_inmap,
    df_fuel_costs=df_mp8_IRA_fuelCosts,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='AEO2023 Reference Case',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )


print(f"""  
====================================================================================================================================================================
PRIVATE PERSPECTIVE COSTS AND BENEFITS: AEO2023 Reference Case
====================================================================================================================================================================
calculate_lifetime_private_impact.py file contains the definition for the calculate_private_NPV function.
Additional information on fuel price lookups can be found in the calculate_fuel_costs.py file. 

def calculate_private_NPV(
        df: pd.DataFrame,
        df_fuel_costs: pd.DataFrame,
        df_baseline_costs: pd.DataFrame,
        input_mp: str,
        menu_mp: int,
        policy_scenario: str,
        discounting_method: str = 'private_fixed',
        base_year: int = 2024,
        verbose: bool = True
) -> pd.DataFrame:

DATAFRAME FOR MP8 AFTER CALCULATING PRIVATE NPV: df_euss_am_mp8_home

AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

-- Scenario: Inflation Reduction Act (IRA) Reference -- 
              scenario_prefix: 'iraRef_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_IRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_iraRef'
              

Determining lifetime private impacts for category: heating with lifetime: 15
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)

Calculating costs for heating...
          input_mp: upgrade08, menu_mp: 8, policy_scenario: AEO2023 Reference Case
Calculating Private NPV for heating with lifetime: 15 years
          policy_scenario: AEO2023 Reference Case --> scenario_prefix: iraRef_mp8_
  

In [34]:
# from cmu_tare_model.adoption_potential.determine_adoption_potential_sensitivity import *

# def adoption_decision(df: pd.DataFrame,
#                       menu_mp: int,
#                       policy_scenario: str,
#                       rcm_model: str,
#                       cr_function: str,
#                       climate_sensitivity: bool = False  # Default is false because we use $190USD2020/mt in Joseph et al. (2025)
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='AP2',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='AP2',
    cr_function='h6c',
    climate_sensitivity=False
    )

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='EASIUR',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='EASIUR',
    cr_function='h6c',
    climate_sensitivity=False
    )

# ============ InMAP =============
df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='InMAP',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='InMAP',
    cr_function='h6c',
    climate_sensitivity=False
    )


print(f"""
====================================================================================================================================================================
ADOPTION FEASIBILITY OF VARIOUS RETROFITS: AEO2023 Reference Case
====================================================================================================================================================================
determine_adoption_potential.py file contains the definition for the adoption_decision function.

DATAFRAME FOR MP8 AFTER DETERMINING ADOPTION FEASIBILITY: df_euss_am_mp8_home
      
AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

-- Scenario: Inflation Reduction Act (IRA) Reference -- 
              scenario_prefix: 'iraRef_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_IRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_iraRef'
              

Calculating Adoption Potential for heating under 'AEO2023 Reference Case' Scenario...
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)
Found 12266 valid homes out of 15651 for heating adoption potential

 --- Public NPV Sensitivity ---
                              Climate Impact Sensitivity:
                                SCC Assumption (Bound): upper
                
               

In [35]:
print(f"Adoption potential for AP2: {df_euss_am_mp8_home_ap2.columns.to_list()}")
print(f"Adoption potential for EASIUR: {df_euss_am_mp8_home_easiur.columns.to_list()}")
print(f"Adoption potential for InMAP: {df_euss_am_mp8_home_inmap.columns.to_list()}")


Adoption potential for AP2: ['square_footage', 'census_region', 'census_division', 'census_division_recs', 'building_america_climate_zone', 'reeds_balancing_area', 'gea_region', 'state', 'city', 'county', 'county_fips', 'puma', 'county_and_puma', 'weather_file_city', 'Longitude', 'Latitude', 'building_type', 'income', 'federal_poverty_level', 'occupancy', 'tenure', 'vacancy_status', 'base_heating_fuel', 'heating_type', 'hvac_cooling_type', 'vintage', 'base_heating_efficiency', 'base_electricity_heating_consumption', 'base_fuelOil_heating_consumption', 'base_naturalGas_heating_consumption', 'base_propane_heating_consumption', 'base_waterHeating_fuel', 'waterHeating_type', 'base_electricity_waterHeating_consumption', 'base_fuelOil_waterHeating_consumption', 'base_naturalGas_waterHeating_consumption', 'base_propane_waterHeating_consumption', 'base_clothesDrying_fuel', 'base_electricity_clothesDrying_consumption', 'base_naturalGas_clothesDrying_consumption', 'base_propane_clothesDrying_con

In [36]:
df_euss_am_mp8_home_ap2

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,iraRef_mp8_clothesDrying_total_npv_lessWTP_upper_AP2_h6c,iraRef_mp8_clothesDrying_total_npv_moreWTP_upper_AP2_h6c,iraRef_mp8_clothesDrying_adoption_upper_AP2_h6c,iraRef_mp8_clothesDrying_impact_upper_AP2_h6c,iraRef_mp8_cooking_health_sensitivity,iraRef_mp8_cooking_benefit_upper_AP2_h6c,iraRef_mp8_cooking_total_npv_lessWTP_upper_AP2_h6c,iraRef_mp8_cooking_total_npv_moreWTP_upper_AP2_h6c,iraRef_mp8_cooking_adoption_upper_AP2_h6c,iraRef_mp8_cooking_impact_upper_AP2_h6c
bldg_id,,,,,,,,,,,,,,,,,,,,,
119,2152.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,-890.58,417.40,Tier 2: Feasible vs. Alternative,Public Benefit,"AP2, h6c",0.00,-1808.83,-736.29,Tier 4: Averse,Public Detriment
122,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,In another census Place,G4200450,...,-267.76,726.30,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
150,1690.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,In another census Place,G4201190,...,-686.59,307.47,Tier 3: Subsidy-Dependent Feasibility,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
153,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200170,...,NaN,NaN,N/A: Invalid Baseline Fuel/Tech,N/A: Invalid Baseline Fuel/Tech,NaN,NaN,NaN,NaN,NaN,NaN
162,2663.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,-75.64,918.42,Tier 3: Subsidy-Dependent Feasibility,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549882,1202.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Not in a census Place,G4200110,...,492.82,1486.88,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
549915,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Lancaster,G4200710,...,78.18,1072.24,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
549937,885.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,115,RFCWc,PA,In another census Place,G4200050,...,-3562.33,-2254.35,Tier 4: Averse,Public Benefit,"AP2, h6c",23.50,-2976.90,-1904.36,Tier 4: Averse,Public Benefit


In [37]:
df_euss_am_mp8_home_easiur

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,iraRef_mp8_clothesDrying_total_npv_lessWTP_upper_EASIUR_h6c,iraRef_mp8_clothesDrying_total_npv_moreWTP_upper_EASIUR_h6c,iraRef_mp8_clothesDrying_adoption_upper_EASIUR_h6c,iraRef_mp8_clothesDrying_impact_upper_EASIUR_h6c,iraRef_mp8_cooking_health_sensitivity,iraRef_mp8_cooking_benefit_upper_EASIUR_h6c,iraRef_mp8_cooking_total_npv_lessWTP_upper_EASIUR_h6c,iraRef_mp8_cooking_total_npv_moreWTP_upper_EASIUR_h6c,iraRef_mp8_cooking_adoption_upper_EASIUR_h6c,iraRef_mp8_cooking_impact_upper_EASIUR_h6c
bldg_id,,,,,,,,,,,,,,,,,,,,,
119,2152.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,-788.38,519.60,Tier 2: Feasible vs. Alternative,Public Benefit,"EASIUR, h6c",0.00,-1646.50,-573.96,Tier 4: Averse,Public Benefit
122,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,In another census Place,G4200450,...,-251.24,742.82,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
150,1690.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,In another census Place,G4201190,...,-645.89,348.17,Tier 3: Subsidy-Dependent Feasibility,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
153,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200170,...,NaN,NaN,N/A: Invalid Baseline Fuel/Tech,N/A: Invalid Baseline Fuel/Tech,NaN,NaN,NaN,NaN,NaN,NaN
162,2663.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,-16.22,977.84,Tier 3: Subsidy-Dependent Feasibility,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549882,1202.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Not in a census Place,G4200110,...,518.87,1512.93,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
549915,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Lancaster,G4200710,...,129.92,1123.98,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
549937,885.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,115,RFCWc,PA,In another census Place,G4200050,...,-3474.36,-2166.38,Tier 4: Averse,Public Benefit,"EASIUR, h6c",86.47,-2913.93,-1841.39,Tier 4: Averse,Public Benefit


In [38]:
df_euss_am_mp8_home_inmap

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,iraRef_mp8_clothesDrying_total_npv_lessWTP_upper_InMAP_h6c,iraRef_mp8_clothesDrying_total_npv_moreWTP_upper_InMAP_h6c,iraRef_mp8_clothesDrying_adoption_upper_InMAP_h6c,iraRef_mp8_clothesDrying_impact_upper_InMAP_h6c,iraRef_mp8_cooking_health_sensitivity,iraRef_mp8_cooking_benefit_upper_InMAP_h6c,iraRef_mp8_cooking_total_npv_lessWTP_upper_InMAP_h6c,iraRef_mp8_cooking_total_npv_moreWTP_upper_InMAP_h6c,iraRef_mp8_cooking_adoption_upper_InMAP_h6c,iraRef_mp8_cooking_impact_upper_InMAP_h6c
bldg_id,,,,,,,,,,,,,,,,,,,,,
119,2152.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,-681.77,626.21,Tier 2: Feasible vs. Alternative,Public Benefit,"InMAP, h6c",0.00,-1428.96,-356.42,Tier 4: Averse,Public Benefit
122,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,In another census Place,G4200450,...,-294.54,699.52,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
150,1690.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,In another census Place,G4201190,...,-579.71,414.35,Tier 3: Subsidy-Dependent Feasibility,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
153,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200170,...,NaN,NaN,N/A: Invalid Baseline Fuel/Tech,N/A: Invalid Baseline Fuel/Tech,NaN,NaN,NaN,NaN,NaN,NaN
162,2663.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,-171.77,822.29,Tier 3: Subsidy-Dependent Feasibility,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549882,1202.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Not in a census Place,G4200110,...,609.99,1604.05,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
549915,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Lancaster,G4200710,...,169.86,1163.92,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
549937,885.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,115,RFCWc,PA,In another census Place,G4200050,...,-3544.72,-2236.74,Tier 4: Averse,Public Benefit,"InMAP, h6c",27.40,-2973.00,-1900.46,Tier 4: Averse,Public Benefit


# Model Runtime

In [39]:
# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

The code took 3 minutes and 32 seconds to execute.


In [40]:
df_euss_am_mp8_home_easiur

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,iraRef_mp8_clothesDrying_total_npv_lessWTP_upper_EASIUR_h6c,iraRef_mp8_clothesDrying_total_npv_moreWTP_upper_EASIUR_h6c,iraRef_mp8_clothesDrying_adoption_upper_EASIUR_h6c,iraRef_mp8_clothesDrying_impact_upper_EASIUR_h6c,iraRef_mp8_cooking_health_sensitivity,iraRef_mp8_cooking_benefit_upper_EASIUR_h6c,iraRef_mp8_cooking_total_npv_lessWTP_upper_EASIUR_h6c,iraRef_mp8_cooking_total_npv_moreWTP_upper_EASIUR_h6c,iraRef_mp8_cooking_adoption_upper_EASIUR_h6c,iraRef_mp8_cooking_impact_upper_EASIUR_h6c
bldg_id,,,,,,,,,,,,,,,,,,,,,
119,2152.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,-788.38,519.60,Tier 2: Feasible vs. Alternative,Public Benefit,"EASIUR, h6c",0.00,-1646.50,-573.96,Tier 4: Averse,Public Benefit
122,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,In another census Place,G4200450,...,-251.24,742.82,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
150,1690.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,In another census Place,G4201190,...,-645.89,348.17,Tier 3: Subsidy-Dependent Feasibility,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
153,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200170,...,NaN,NaN,N/A: Invalid Baseline Fuel/Tech,N/A: Invalid Baseline Fuel/Tech,NaN,NaN,NaN,NaN,NaN,NaN
162,2663.0,Northeast,Middle Atlantic,Middle Atlantic,Mixed-Humid,122,RFCEc,PA,Not in a census Place,G4200450,...,-16.22,977.84,Tier 3: Subsidy-Dependent Feasibility,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549882,1202.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Not in a census Place,G4200110,...,518.87,1512.93,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
549915,2176.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,122,RFCEc,PA,Lancaster,G4200710,...,129.92,1123.98,Tier 2: Feasible vs. Alternative,Public Benefit,NaN,NaN,NaN,NaN,NaN,NaN
549937,885.0,Northeast,Middle Atlantic,Middle Atlantic,Cold,115,RFCWc,PA,In another census Place,G4200050,...,-3474.36,-2166.38,Tier 4: Averse,Public Benefit,"EASIUR, h6c",86.47,-2913.93,-1841.39,Tier 4: Averse,Public Benefit


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from typing import List, Optional, Tuple, Dict, Any, Union

from cmu_tare_model.adoption_potential.data_processing.visuals_adoption_potential_utils import (
    verify_columns_exist,
    detect_adoption_columns,
    filter_columns,
    filter_by_fuel,
    plot_adoption_rate_bar
)

# =========================================================================
# FUNCTIONS: VISUALIZATION USING DATAFRAMES AND SUBPLOTS
# =========================================================================

# def create_df_adoption(
#         df: pd.DataFrame, 
#         menu_mp: int, 
#         category: str, 
#         scc: str = 'upper', 
#         rcm_model: str = 'AP2', 
#         cr_function: str = 'acs', 
#         mer_type: str = 'lrmer'  # Kept for backward compatibility
# ) -> pd.DataFrame:
#     """
#     Generates a new DataFrame with specific adoption columns.
    
#     Args:
#         df: Original DataFrame
#         menu_mp: Measure package identifier
#         category: Equipment category (e.g., 'heating', 'waterHeating')
#         scc: Social cost of carbon assumption ('lower', 'central', or 'upper')
#         rcm_model: RCM model ('AP2', 'EASIUR', or 'InMAP')
#         cr_function: Concentration-response function ('acs' or 'h6c')
#         mer_type: Marginal emission rate type ('lrmer' or 'srmer')
#               Kept for backward compatibility with old column naming pattern

#     Returns:
#         DataFrame with selected columns
#     """
#     # Create a copy of the dataframe
#     df_copy = df.copy()
    
#     print(f"\nCreating adoption DataFrame for {category}:")
#     print(f"- menu_mp: {menu_mp}")
#     print(f"- scc: {scc}, rcm_model: {rcm_model}, cr_function: {cr_function}")
    
#     # Define required columns in the desired order
#     required_cols = [
#         f'base_{category}_fuel',
#         f'include_{category}',
#         'percent_AMI',
#         'lowModerateIncome_designation',
#         f'preIRA_mp{menu_mp}_{category}_public_npv_{scc}_{rcm_model}_{cr_function}',    # PRE IRA REFERENCE CASE SCENARIO
#         f'preIRA_mp{menu_mp}_{category}_total_capitalCost', 
#         f'preIRA_mp{menu_mp}_{category}_private_npv_lessWTP',
#         f'preIRA_mp{menu_mp}_{category}_total_npv_lessWTP_{scc}_{rcm_model}_{cr_function}',
#         f'preIRA_mp{menu_mp}_{category}_net_capitalCost',
#         f'preIRA_mp{menu_mp}_{category}_private_npv_moreWTP',
#         f'preIRA_mp{menu_mp}_{category}_total_npv_moreWTP_{scc}_{rcm_model}_{cr_function}',
#         f'preIRA_mp{menu_mp}_{category}_adoption_{scc}_{rcm_model}_{cr_function}',
#         f'mp{menu_mp}_{category}_rebate_amount',    # IRA REFERENCE CASE SCENARIO
#         f'iraRef_mp{menu_mp}_{category}_public_npv_{scc}_{rcm_model}_{cr_function}',
#         f'iraRef_mp{menu_mp}_{category}_total_capitalCost', 
#         f'iraRef_mp{menu_mp}_{category}_private_npv_lessWTP',
#         f'iraRef_mp{menu_mp}_{category}_total_npv_lessWTP_{scc}_{rcm_model}_{cr_function}',
#         f'iraRef_mp{menu_mp}_{category}_net_capitalCost',
#         f'iraRef_mp{menu_mp}_{category}_private_npv_moreWTP',
#         f'iraRef_mp{menu_mp}_{category}_total_npv_moreWTP_{scc}_{rcm_model}_{cr_function}',
#         f'iraRef_mp{menu_mp}_{category}_adoption_{scc}_{rcm_model}_{cr_function}',
#     ]

#     # Define optional columns (sensitivity analysis)
#     optional_cols = [
#         f'preIRA_mp{menu_mp}_{category}_avoided_mt_co2e_{mer_type}',    
#         f'iraRef_mp{menu_mp}_{category}_avoided_mt_co2e_{mer_type}',
#         f'iraRef_mp{menu_mp}_{category}_usd2023_per_mtCO2e_{mer_type}',
#         f'iraRef_mp{menu_mp}_{category}_benefit_{scc}_{rcm_model}_{cr_function}',
#     ]

#     try:
#         # Use verify_columns_exist for consistent column checking
#         existing_cols = verify_columns_exist(
#             df=df_copy,
#             required_cols=required_cols,
#             optional_cols=optional_cols
#             )
        
#         print(f"- Found {len(existing_cols)} columns")
            
#         # Select the relevant columns in the original order
#         result = df_copy[existing_cols]
#         print(f"- Result shape: {result.shape}")
#         return result
    
#     except Exception as e:
#         print(f"! Error creating adoption DataFrame: {str(e)}")
#         return df.copy()


def create_df_adoption(
        df: pd.DataFrame, 
        menu_mp: int, 
        category: str, 
        scc: str = 'upper', 
        rcm_model: str = 'AP2', 
        cr_function: str = 'acs', 
        mer_type: str = 'lrmer'  # Kept for backward compatibility
) -> pd.DataFrame:
    """
    Generates a new DataFrame with specific adoption columns.
    
    Args:
        df: Original DataFrame
        menu_mp: Measure package identifier
        category: Equipment category (e.g., 'heating', 'waterHeating')
        scc: Social cost of carbon assumption ('lower', 'central', or 'upper')
        rcm_model: RCM model ('AP2', 'EASIUR', or 'InMAP')
        cr_function: Concentration-response function ('acs' or 'h6c')
        mer_type: Marginal emission rate type ('lrmer' or 'srmer')
              Kept for backward compatibility with old column naming pattern

    Returns:
        DataFrame with selected columns
    """
    # Create a copy of the dataframe
    df_copy = df.copy()
    
    print(f"\nCreating adoption DataFrame for {category}:")
    print(f"- menu_mp: {menu_mp}")
    print(f"- scc: {scc}, rcm_model: {rcm_model}, cr_function: {cr_function}")
    
    # Define required columns in the desired order
    required_cols = [
        f'base_{category}_fuel',
        f'include_{category}',
        'percent_AMI',
        'lowModerateIncome_designation',
        f'preIRA_mp{menu_mp}_{category}_public_npv_{scc}_{rcm_model}_{cr_function}',    # PRE IRA REFERENCE CASE SCENARIO
        f'preIRA_mp{menu_mp}_{category}_total_capitalCost', 
        f'preIRA_mp{menu_mp}_{category}_private_npv_lessWTP',
        f'preIRA_mp{menu_mp}_{category}_total_npv_lessWTP_{scc}_{rcm_model}_{cr_function}',
        f'preIRA_mp{menu_mp}_{category}_net_capitalCost',
        f'preIRA_mp{menu_mp}_{category}_private_npv_moreWTP',
        f'preIRA_mp{menu_mp}_{category}_total_npv_moreWTP_{scc}_{rcm_model}_{cr_function}',
        f'preIRA_mp{menu_mp}_{category}_adoption_{scc}_{rcm_model}_{cr_function}',
        f'mp{menu_mp}_{category}_rebate_amount',    # IRA REFERENCE CASE SCENARIO
        f'iraRef_mp{menu_mp}_{category}_public_npv_{scc}_{rcm_model}_{cr_function}',
        f'iraRef_mp{menu_mp}_{category}_total_capitalCost', 
        f'iraRef_mp{menu_mp}_{category}_private_npv_lessWTP',
        f'iraRef_mp{menu_mp}_{category}_total_npv_lessWTP_{scc}_{rcm_model}_{cr_function}',
        f'iraRef_mp{menu_mp}_{category}_net_capitalCost',
        f'iraRef_mp{menu_mp}_{category}_private_npv_moreWTP',
        f'iraRef_mp{menu_mp}_{category}_total_npv_moreWTP_{scc}_{rcm_model}_{cr_function}',
        f'iraRef_mp{menu_mp}_{category}_adoption_{scc}_{rcm_model}_{cr_function}',
    ]

    # Define optional columns (sensitivity analysis)
    optional_cols = [
        f'preIRA_mp{menu_mp}_{category}_avoided_mt_co2e_{mer_type}',    
        f'iraRef_mp{menu_mp}_{category}_avoided_mt_co2e_{mer_type}',
        f'iraRef_mp{menu_mp}_{category}_usd2023_per_mtCO2e_{mer_type}',
        f'iraRef_mp{menu_mp}_{category}_benefit_{scc}_{rcm_model}_{cr_function}',
    ]

    try:
        # Use verify_columns_exist for consistent column checking
        existing_cols = verify_columns_exist(
            df=df_copy,
            required_cols=required_cols,
            optional_cols=optional_cols
            )
        
        print(f"- Found {len(existing_cols)} columns")
            
        # Select the relevant columns in the original order
        result = df_copy[existing_cols]
        print(f"- Result shape: {result.shape}")
        return result
    
    except Exception as e:
        print(f"! Error creating adoption DataFrame: {str(e)}")
        return df.copy()
    

def create_multiIndex_adoption_df(
        df: pd.DataFrame, 
        menu_mp: int, 
        category: str, 
        scc: str = 'upper', 
        rcm_model: str = 'AP2', 
        cr_function: str = 'acs',
        mer_type: str = 'lrmer'  # Kept for backward compatibility
) -> pd.DataFrame:
    """
    Creates a multi-index DataFrame showing adoption percentages by tier, fuel type, and income level.
    
    Args:
        df: DataFrame containing adoption data
        menu_mp: Measure package identifier
        category: Equipment category (e.g., 'heating', 'waterHeating')
        scc: Social cost of carbon assumption
        rcm_model: RCM model 
        cr_function: Concentration-response function
        mer_type: Marginal emission rate type (kept for backward compatibility)
        
    Returns:
        A filtered multi-index DataFrame with adoption percentages
    """
    # Make a copy to avoid modifying the original
    df_processed = df.copy()
    
    print(f"\nCreating multi-index adoption DataFrame for {category}:")
    print(f"- menu_mp: {menu_mp}")
    print(f"- scc: {scc}, rcm_model: {rcm_model}, cr_function: {cr_function}")
    print(f"- Input DataFrame shape: {df_processed.shape}")
    
    try:
        # Define required columns and verify they exist
        fuel_col = f'base_{category}_fuel'
        verify_columns_exist(df_processed, [fuel_col, 'lowModerateIncome_designation'])
        
        # Get adoption columns (will raise ValueError if not found)
        adoption_cols = detect_adoption_columns(
            df_processed, menu_mp, category, scc, rcm_model, cr_function
        )
        
        # Set 'lowModerateIncome_designation' as categorical with order
        income_categories = ['Low-Income', 'Moderate-Income', 'Middle-to-Upper-Income']            
        df_processed['lowModerateIncome_designation'] = pd.Categorical(
            df_processed['lowModerateIncome_designation'], 
            categories=income_categories, 
            ordered=True
        )
        
        # Print value counts of adoption columns for debugging
        print(f"\nAdoption column value counts for {category}:")
        for col in adoption_cols:
            print(f"\n{col}:")
            value_counts = df_processed[col].value_counts().to_dict()
            for value, count in value_counts.items():
                print(f"  {value}: {count} ({count/len(df_processed)*100:.1f}%)")
        
        # Group by fuel and income, calculate normalized counts
        print("\nGrouping and calculating percentages...")
        percentages_df = df_processed.groupby(
            [fuel_col, 'lowModerateIncome_designation'], 
            observed=False
        )[adoption_cols].apply(
            lambda x: x.apply(lambda y: y.value_counts(normalize=True))
        ).unstack().fillna(0) * 100
        
        print(f"- Grouped DataFrame shape: {percentages_df.shape}")
        
        # Round for readability
        percentages_df = percentages_df.round(0)
        
        # Ensure tier columns exist and calculate totals
        print("\nEnsuring all tier columns exist and calculating totals...")
        
        for column in adoption_cols:
            for tier in ['Tier 1: Feasible', 'Tier 2: Feasible vs. Alternative', 
                        'Tier 3: Subsidy-Dependent Feasibility']:
                if (column, tier) not in percentages_df.columns:
                    print(f"- Adding missing tier column: ({column}, {tier})")
                    percentages_df[(column, tier)] = 0
                    
            # Calculate totals
            percentages_df[(column, 'Total Adoption Potential')] = (
                percentages_df[(column, 'Tier 1: Feasible')] + 
                percentages_df[(column, 'Tier 2: Feasible vs. Alternative')]
            )
            
            percentages_df[(column, 'Total Adoption Potential (Additional Subsidy)')] = (
                percentages_df[(column, 'Tier 1: Feasible')] + 
                percentages_df[(column, 'Tier 2: Feasible vs. Alternative')] + 
                percentages_df[(column, 'Tier 3: Subsidy-Dependent Feasibility')]
            )
            
        # Rebuild MultiIndex and filter
        percentages_df.columns = pd.MultiIndex.from_tuples(percentages_df.columns)
        
        print("\nFiltering and sorting columns...")
        filtered_df = filter_columns(percentages_df)
        print(f"- Final DataFrame shape: {filtered_df.shape}")
        
        # Sort the DataFrame appropriately  
        if not filtered_df.empty:
            # Sort by index levels if they exist
            index_levels = [level for level in filtered_df.index.names if level is not None]
            if index_levels:
                try:
                    filtered_df = filtered_df.sort_index(level=index_levels)
                    print("- DataFrame sorted by index levels")
                except Exception as e:
                    print(f"! Warning: Could not sort DataFrame: {str(e)}")
                
        return filtered_df
        
    except Exception as e:
        print(f"! Error processing adoption data: {str(e)}")
        return pd.DataFrame()  # Return empty DataFrame on error
    

# def subplot_grid_adoption_vBar(
#         dataframes: List[pd.DataFrame],
#         subplot_positions: List[Tuple[int, int]],
#         categories: Optional[List[str]] = None,
#         menu_mp: Optional[int] = None,
#         scenarios_list: Optional[List[List[str]]] = None,
#         scc: str = 'upper',
#         rcm_model: str = 'AP2',
#         cr_function: str = 'acs',
#         filter_fuel: Optional[List[str]] = None,
#         x_labels: Optional[List[str]] = None,
#         plot_titles: Optional[List[str]] = None,
#         y_labels: Optional[List[str]] = None,
#         suptitle: Optional[str] = None,
#         figure_size: Tuple[int, int] = (12, 10),
#         sharex: bool = False,
#         sharey: bool = False
# ) -> None:
#     """
#     Creates a grid of subplots to visualize adoption rates across different categories.

#     Args:
#         dataframes: List of DataFrames with adoption data
#         subplot_positions: Positions of subplots as (row, col) tuples
#         categories: List of equipment categories (e.g., ['heating', 'waterHeating'])
#                    Required if scenarios_list is not provided.
#         menu_mp: Measure package identifier
#                  Required if scenarios_list is not provided.
#         scenarios_list: List of lists of column names for each DataFrame 
#                       If provided, overrides categories and menu_mp.
#         scc: Social cost of carbon assumption ('lower', 'central', or 'upper')
#         rcm_model: RCM model ('AP2', 'EASIUR', or 'InMAP')
#         cr_function: Concentration-response function ('acs' or 'h6c')
#         filter_fuel: List of fuel types to filter by
#         x_labels: Labels for the x-axis of each subplot
#         plot_titles: Titles for each subplot
#         y_labels: Labels for the y-axis of each subplot
#         suptitle: Central title for the entire figure
#         figure_size: Size of the figure as (width, height)
#         sharex: Whether subplots should share the same x-axis
#         sharey: Whether subplots should share the same y-axis
#     """
#     print(f"\nCreating subplot grid for adoption visualization:")
#     print(f"- Number of dataframes: {len(dataframes)}")
#     print(f"- Number of subplot positions: {len(subplot_positions)}")
#     if categories:
#         print(f"- Categories: {categories}")
#     print(f"- menu_mp: {menu_mp}")
#     print(f"- scc: {scc}, rcm_model: {rcm_model}, cr_function: {cr_function}")
#     if filter_fuel:
#         print(f"- Filtering for fuel types: {filter_fuel}")
    
#     # Define color mapping for legend
#     color_mapping = {
#         'Tier 1: Feasible': 'steelblue',
#         'Tier 2: Feasible vs. Alternative': 'lightblue',
#         'Tier 3: Subsidy-Dependent Feasibility': 'lightsalmon'
#     }

#     # Validate inputs
#     if not dataframes or len(dataframes) != len(subplot_positions):
#         print("! Error: Invalid input parameters. Check dataframes and positions.")
#         return
        
#     # Derive scenarios from categories if scenarios_list not provided
#     derived_scenarios_list = []
#     if scenarios_list is None:
#         if categories is None or menu_mp is None:
#             print("! Error: Must provide either scenarios_list or both categories and menu_mp.")
#             return
            
#         if len(categories) != len(dataframes):
#             print("! Warning: Length of categories doesn't match length of dataframes.")
#             if len(categories) < len(dataframes):
#                 print(f"  Truncating to {len(categories)} dataframes")
#                 dataframes = dataframes[:len(categories)]
#                 subplot_positions = subplot_positions[:len(categories)]
#             else:
#                 print(f"  Using only {len(dataframes)} categories")
#                 categories = categories[:len(dataframes)]
            
#         # Derive scenarios for each DataFrame
#         for df, category in zip(dataframes, categories):
#             try:
#                 adoption_cols = detect_adoption_columns(
#                     df, menu_mp, category, scc, rcm_model, cr_function
#                 )
#                 derived_scenarios_list.append(adoption_cols)
#                 print(f"- Detected adoption columns for {category}: {adoption_cols}")
#             except Exception as e:
#                 print(f"! Error detecting adoption columns for {category}: {str(e)}")
#                 derived_scenarios_list.append([])
        
#         # Use derived scenarios
#         scenarios_list = derived_scenarios_list
    
#     # Ensure scenarios_list matches dataframes length
#     if len(scenarios_list) != len(dataframes):
#         print("! Warning: Length of scenarios_list doesn't match length of dataframes.")
#         min_len = min(len(scenarios_list), len(dataframes))
#         dataframes = dataframes[:min_len]
#         subplot_positions = subplot_positions[:min_len]
#         scenarios_list = scenarios_list[:min_len]
#         if categories:
#             categories = categories[:min_len]
    
#     # Determine grid dimensions
#     num_cols = max(pos[1] for pos in subplot_positions) + 1
#     num_rows = max(pos[0] for pos in subplot_positions) + 1
    
#     print(f"- Creating {num_rows}x{num_cols} subplot grid")

#     # Create figure and axes
#     fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=figure_size, 
#                            sharex=sharex, sharey=sharey)
    
#     # Handle single subplot case
#     if num_rows == 1 and num_cols == 1:
#         axes = np.array([[axes]])
#     elif num_rows == 1:
#         axes = np.array([axes])
#     elif num_cols == 1:
#         axes = np.array([[ax] for ax in axes])
        
#     # Process each subplot
#     for idx, (df, scenarios) in enumerate(zip(dataframes, scenarios_list)):
#         # Skip if no scenarios found
#         if not scenarios:
#             print(f"! Warning: No scenarios for DataFrame {idx}. Skipping.")
#             continue
            
#         # Apply fuel filter if provided
#         if filter_fuel and not df.empty:
#             df = filter_by_fuel(df, filter_fuel)
            
#         # Skip if filtered DataFrame is empty
#         if df.empty:
#             print(f"! Warning: No data after fuel filtering for DataFrame {idx}. Skipping.")
#             continue
            
#         try:
#             # Get subplot position and configure
#             pos = subplot_positions[idx]
#             ax = axes[pos[0], pos[1]]
            
#             # Set labels if provided
#             x_label = x_labels[idx] if x_labels and idx < len(x_labels) else ""
#             y_label = y_labels[idx] if y_labels and idx < len(y_labels) else ""
            
#             # Set title - use category if available, otherwise use provided title
#             if plot_titles and idx < len(plot_titles):
#                 title = plot_titles[idx]
#             elif categories and idx < len(categories):
#                 title = categories[idx].capitalize()
#             else:
#                 title = f"Plot {idx+1}"

#             # Plot the data
#             plot_adoption_rate_bar(df, scenarios, title, x_label, y_label, ax)
            
#         except Exception as e:
#             print(f"! Error plotting at position {pos}: {str(e)}")
#             # Get subplot position
#             pos = subplot_positions[idx]
#             ax = axes[pos[0], pos[1]]
#             # Display error message on plot
#             ax.text(0.5, 0.5, f"Error: {str(e)}", ha='center', va='center', 
#                   transform=ax.transAxes, color='red')

#     # Add title and legend
#     if suptitle:
#         plt.suptitle(suptitle, fontweight='bold', fontsize=20)

#     # Create legend
#     legend_labels = list(color_mapping.keys())
#     legend_handles = [plt.Rectangle((0, 0), 1, 1, color=color_mapping[label]) 
#                     for label in legend_labels]
            
#     fig.legend(legend_handles, legend_labels, loc='lower center', 
#              ncol=len(legend_labels), prop={'size': 20}, 
#              labelspacing=0.5, bbox_to_anchor=(0.5, -0.05))

#     # Adjust layout and display
#     plt.tight_layout(rect=[0, 0.03, 1, 0.95])
#     plt.show()


def subplot_grid_adoption_vBar(
        dataframes: List[pd.DataFrame],
        subplot_positions: List[Tuple[int, int]],
        categories: Optional[List[str]] = None,
        menu_mp: Optional[int] = None,
        scenarios_list: Optional[List[List[str]]] = None,
        scc: str = 'upper',
        rcm_model: str = 'AP2',
        cr_function: str = 'acs',
        filter_fuel: Optional[List[str]] = None,
        x_labels: Optional[List[str]] = None,
        plot_titles: Optional[List[str]] = None,
        y_labels: Optional[List[str]] = None,
        suptitle: Optional[str] = None,
        figure_size: Tuple[int, int] = (12, 10),
        sharex: bool = False,
        sharey: bool = False
) -> None:
    """
    Creates a grid of subplots to visualize adoption rates across different categories.

    Args:
        dataframes: List of DataFrames with adoption data
        subplot_positions: Positions of subplots as (row, col) tuples
        categories: List of equipment categories (e.g., ['heating', 'waterHeating'])
                   Required if scenarios_list is not provided.
        menu_mp: Measure package identifier
                 Required if scenarios_list is not provided.
        scenarios_list: List of lists of column names for each DataFrame 
                      If provided, overrides categories and menu_mp.
        scc: Social cost of carbon assumption ('lower', 'central', or 'upper')
        rcm_model: RCM model ('AP2', 'EASIUR', or 'InMAP')
        cr_function: Concentration-response function ('acs' or 'h6c')
        filter_fuel: List of fuel types to filter by
        x_labels: Labels for the x-axis of each subplot
        plot_titles: Titles for each subplot
        y_labels: Labels for the y-axis of each subplot
        suptitle: Central title for the entire figure
        figure_size: Size of the figure as (width, height)
        sharex: Whether subplots should share the same x-axis
        sharey: Whether subplots should share the same y-axis
    """
    print(f"\nCreating subplot grid for adoption visualization:")
    print(f"- Number of dataframes: {len(dataframes)}")
    print(f"- Number of subplot positions: {len(subplot_positions)}")
    if categories:
        print(f"- Categories: {categories}")
    print(f"- menu_mp: {menu_mp}")
    print(f"- scc: {scc}, rcm_model: {rcm_model}, cr_function: {cr_function}")
    if filter_fuel:
        print(f"- Filtering for fuel types: {filter_fuel}")
    
    # Define color mapping for legend
    color_mapping = {
        'Tier 1: Feasible': 'steelblue',
        'Tier 2: Feasible vs. Alternative': 'lightblue',
        'Tier 3: Subsidy-Dependent Feasibility': 'lightsalmon'
    }

    # Validate inputs
    if not dataframes or len(dataframes) != len(subplot_positions):
        print("! Error: Invalid input parameters. Check dataframes and positions.")
        return
        
    # Derive scenarios from categories if scenarios_list not provided
    derived_scenarios_list = []
    if scenarios_list is None:
        if categories is None or menu_mp is None:
            print("! Error: Must provide either scenarios_list or both categories and menu_mp.")
            return
            
        if len(categories) != len(dataframes):
            print("! Warning: Length of categories doesn't match length of dataframes.")
            if len(categories) < len(dataframes):
                print(f"  Truncating to {len(categories)} dataframes")
                dataframes = dataframes[:len(categories)]
                subplot_positions = subplot_positions[:len(categories)]
            else:
                print(f"  Using only {len(dataframes)} categories")
                categories = categories[:len(dataframes)]
            
        # Derive scenarios for each DataFrame - KEY CHANGE HERE
        for df, category in zip(dataframes, categories):
            try:
                # Pass through the sensitivity parameters instead of using defaults
                adoption_cols = detect_adoption_columns(
                    df, menu_mp, category, scc, rcm_model, cr_function
                )
                derived_scenarios_list.append(adoption_cols)
                print(f"- Detected adoption columns for {category}: {adoption_cols}")
            except Exception as e:
                print(f"! Error detecting adoption columns for {category}: {str(e)}")
                derived_scenarios_list.append([])
        
        # Use derived scenarios
        scenarios_list = derived_scenarios_list
    
    # Rest of the function remains unchanged
    # ...

    
# ========================================================================
# EXAMPLE USAGE
# ========================================================================
# Create adoption DataFrames for various categories
df_heating = create_df_adoption(df_euss_am_mp8_home_easiur, menu_mp=8, category='heating', 
                               rcm_model='EASIUR', cr_function='acs')
df_water = create_df_adoption(df_euss_am_mp8_home_easiur, menu_mp=8, category='waterHeating', 
                             rcm_model='EASIUR', cr_function='acs')

# Convert to multi-index format for visualization
mi_heating = create_multiIndex_adoption_df(df_heating, menu_mp=8, category='heating',
                                         rcm_model='EASIUR', cr_function='acs')
mi_water = create_multiIndex_adoption_df(df_water, menu_mp=8, category='waterHeating',
                                       rcm_model='EASIUR', cr_function='acs')

# Create visualization
subplot_grid_adoption_vBar(
    dataframes=[mi_heating, mi_water],
    subplot_positions=[(0, 0), (0, 1)],
    categories=['heating', 'waterHeating'],
    menu_mp=8,
    rcm_model='EASIUR',  # Now correctly passed to detect_adoption_columns
    suptitle='Adoption Potential by Tier',
    filter_fuel=['Electricity', 'Natural Gas']
)


Creating adoption DataFrame for heating:
- menu_mp: 8
- scc: upper, rcm_model: EASIUR, cr_function: acs
- Found 24 columns
- Result shape: (15651, 24)

Creating adoption DataFrame for waterHeating:
- menu_mp: 8
- scc: upper, rcm_model: EASIUR, cr_function: acs
- Found 24 columns
- Result shape: (15651, 24)

Creating multi-index adoption DataFrame for heating:
- menu_mp: 8
- scc: upper, rcm_model: EASIUR, cr_function: acs
- Input DataFrame shape: (15651, 24)
Looking for adoption columns:
  - Pre-IRA: preIRA_mp8_heating_adoption_upper_EASIUR_acs
  - IRA: iraRef_mp8_heating_adoption_upper_EASIUR_acs

Adoption column value counts for heating:

preIRA_mp8_heating_adoption_upper_EASIUR_acs:
  Tier 4: Averse: 5786 (37.0%)
  Tier 3: Subsidy-Dependent Feasibility: 3714 (23.7%)
  N/A: Invalid Baseline Fuel/Tech: 3385 (21.6%)
  Tier 2: Feasible vs. Alternative: 1497 (9.6%)
  Tier 1: Feasible: 1269 (8.1%)

iraRef_mp8_heating_adoption_upper_EASIUR_acs:
  Tier 3: Subsidy-Dependent Feasibility: 4628

In [42]:
# Create basic visualization to investigate tier distribution
heating_df = create_multiIndex_adoption_df(
    df=df_euss_am_mp8_home_easiur, menu_mp=8, category='heating', scc='upper', rcm_model='EASIUR', cr_function='acs'
)

subplot_grid_adoption_vBar(
    dataframes=[heating_df],
    subplot_positions=[(0, 0)],

    categories=['heating'],
    menu_mp=8,
    suptitle="Tier Distribution for Heating"
)




Creating multi-index adoption DataFrame for heating:
- menu_mp: 8
- scc: upper, rcm_model: EASIUR, cr_function: acs
- Input DataFrame shape: (15651, 757)
Looking for adoption columns:
  - Pre-IRA: preIRA_mp8_heating_adoption_upper_EASIUR_acs
  - IRA: iraRef_mp8_heating_adoption_upper_EASIUR_acs

Adoption column value counts for heating:

preIRA_mp8_heating_adoption_upper_EASIUR_acs:
  Tier 4: Averse: 5786 (37.0%)
  Tier 3: Subsidy-Dependent Feasibility: 3714 (23.7%)
  N/A: Invalid Baseline Fuel/Tech: 3385 (21.6%)
  Tier 2: Feasible vs. Alternative: 1497 (9.6%)
  Tier 1: Feasible: 1269 (8.1%)

iraRef_mp8_heating_adoption_upper_EASIUR_acs:
  Tier 3: Subsidy-Dependent Feasibility: 4628 (29.6%)
  N/A: Invalid Baseline Fuel/Tech: 3385 (21.6%)
  Tier 4: Averse: 3345 (21.4%)
  Tier 1: Feasible: 2748 (17.6%)
  Tier 2: Feasible vs. Alternative: 1545 (9.9%)

Grouping and calculating percentages...
- Grouped DataFrame shape: (15, 10)

Ensuring all tier columns exist and calculating totals...

Fi

In [43]:
# Create basic visualization to investigate tier distribution
heating_df = create_multiIndex_adoption_df(
    df_basic_adoption_easiur_acs, menu_mp=8, category='heating', scc='upper', rcm_model='AP2', cr_function='acs'
)

waterHeating_df = create_multiIndex_adoption_df(
    df_basic_adoption_easiur_acs, menu_mp=8, category='waterHeating', scc='upper', rcm_model='AP2', cr_function='acs'
)

# Test filtering by specific fuels
subplot_grid_adoption_vBar(
    dataframes=[heating_df, water_df],
    categories=['heating', 'waterHeating'],
    menu_mp=8,
    subplot_positions=[(0, 0), (0, 1)],
    filter_fuel=['Electricity'],
    suptitle="Adoption Potential for Electric Systems Only"
)

NameError: name 'df_basic_adoption_easiur_acs' is not defined

In [ ]:
print(df_basic_adoption_easiur_acs.columns)

In [ ]:
if print_debug:
    # def create_sample_df(
    #     df: pd.DataFrame,
    #     include_groups: Optional[List[str]] = None,
    #     categories: Optional[List[str]] = None,
    #     scenarios: Optional[List[str]] = None,
    #     metrics: Optional[List[str]] = None,
    #     mp_number: int = 8,
    #     regex_patterns: Optional[Union[str, List[str]]] = None  # New parameter
    # ) -> pd.DataFrame:

    # df_sample_climate

    # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home_easiur,
        include_groups=['costs', 'adoption'],
        categories=['heating'],
        scenarios=['baseline', 'preIRA', 'iraRef'],
        # metrics=[],
        mp_number=menu_mp,
        regex_patterns=[
            # 'upgrade_heating',
            # 'valid_fuel_heating',
            'include_heating',
            'heating_private',
            # 'total_npv',
            'heating_adoption'
            ]
    )
    print(df_sample_heating)

    # # Create a sample dataframe for the waterHeating category
    # df_sample_waterHeating = create_sample_df(
    #     df=df_euss_am_mp8_home,
    #     include_groups=['base_equipment'],
    #     categories=['waterHeating'],
    #     scenarios=['preIRA', 'iraRef'],
    #     metrics=[],
    #     mp_number=menu_mp,
    #     regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'include_waterHeating', 'waterHeating_adoption']
    # )
    # print(df_sample_waterHeating)

    # # Create a sample dataframe for the clothesDrying category
    # df_sample_clothesDrying = create_sample_df(
    #     df=df_euss_am_mp8_home,
    #     include_groups=['base_equipment'],
    #     categories=['clothesDrying'],
    #     scenarios=['preIRA', 'iraRef'],
    #     metrics=[],
    #     mp_number=menu_mp,
    #     regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_adoption']
    # )
    # print(df_sample_clothesDrying)

    # # Create a sample dataframe for the cooking category
    # df_sample_cooking = create_sample_df(
    #     df=df_euss_am_mp8_home,
    #     include_groups=['base_equipment'],
    #     categories=['cooking'],
    #     scenarios=['preIRA', 'iraRef'],
    #     metrics=[],
    #     mp_number=menu_mp,
    #     regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_adoption']
    # )
    # print(df_sample_cooking)

In [ ]:
# ============= AP2 Model =============
# ACS Concentration-Response Function
df_multi_ap2_acs = create_multiIndex_adoption_df(
    df=df_basic_adoption_ap2_acs,
    menu_mp=menu_mp,
    category=category,
    scc='upper',
    rcm_model='AP2',
    cr_function='acs'
)

# H6C Concentration-Response Function
df_multi_ap2_h6c = create_multiIndex_adoption_df(
    df=df_basic_adoption_ap2_h6c,
    menu_mp=menu_mp,
    category=category,
    scc='upper',
    rcm_model='AP2',
    cr_function='h6c'
)

# ============= EASIUR Model =============
# ACS Concentration-Response Function
df_multi_easiur_acs = create_multiIndex_adoption_df(
    df=df_basic_adoption_easiur_acs,
    menu_mp=menu_mp,
    category=category,
    scc='upper',
    rcm_model='EASIUR',
    cr_function='acs'
)

# H6C Concentration-Response Function
df_multi_easiur_h6c = create_multiIndex_adoption_df(
    df=df_basic_adoption_easiur_h6c,
    menu_mp=menu_mp,
    category=category,
    scc='upper',
    rcm_model='EASIUR',
    cr_function='h6c'
)

# ============= InMAP Model =============
# ACS Concentration-Response Function
df_multi_inmap_acs = create_multiIndex_adoption_df(
    df=df_basic_adoption_inmap_acs,
    menu_mp=menu_mp,
    category=category,
    scc='upper',
    rcm_model='InMAP',
    cr_function='acs'
)

# H6C Concentration-Response Function
df_multi_inmap_h6c = create_multiIndex_adoption_df(
    df=df_basic_adoption_inmap_h6c,
    menu_mp=menu_mp,
    category=category,
    scc='upper',
    rcm_model='InMAP',
    cr_function='h6c'
)

In [ ]:
print(df_multi_easiur_acs)

In [ ]:
df_multiIndex_basic_heating_adoption_srmer

In [ ]:
df_multiIndex_moderate_heating_adoption_lrmer

In [ ]:
df_multiIndex_moderate_heating_adoption_srmer

In [ ]:
df_multiIndex_advanced_heating_adoption_lrmer

In [ ]:
df_multiIndex_advanced_heating_adoption_srmer

# Water Heating, Clothes Drying, and Cooking

In [ ]:
# Function call remains as before
subplot_grid_adoption_vBar(
    dataframes=[df_multiIndex_basic_waterHeating_adoption_lrmer, 
                df_multiIndex_basic_clothesDrying_adoption_lrmer, 
                df_multiIndex_basic_cooking_adoption_lrmer,
               ],
    scenarios_list = [
        ['preIRA_mp8_waterHeating_adoption_lrmer', 'iraRef_mp8_waterHeating_adoption_lrmer'],
        ['preIRA_mp8_clothesDrying_adoption_lrmer', 'iraRef_mp8_clothesDrying_adoption_lrmer'],
        ['preIRA_mp8_cooking_adoption_lrmer', 'iraRef_mp8_cooking_adoption_lrmer']
    ],
    subplot_positions=[(0, 0), (0, 1), (0, 2),
                    #    (1, 0), (1, 1), (1, 2),
                    #    (2, 0), (2, 1), (2, 2)
                       ],
    x_labels=["", "", "",
              "", "", "",
              "", "Household Income Designation and Baseline Fuel (Pre-Retrofit)", ""
             ],
    plot_titles=["Heat Pump Water Heater:\nPre-IRA vs. IRA-Reference", "Heat Pump Clothes Dryer:\nPre-IRA vs. IRA-Reference", "Electric Resistance Range:\nPre-IRA vs. IRA-Reference"],
    y_labels=["LRMER - Retrofit Adoption Potential (%)", "", "",
             ],
    figure_size=(20,10),
    sharex=False,
    sharey=True,
#     filter_fuel=['Natural Gas', 'Electricity', 'Propane']
)

# Function call remains as before
subplot_grid_adoption_vBar(
    dataframes=[df_multiIndex_basic_waterHeating_adoption_srmer, 
                df_multiIndex_basic_clothesDrying_adoption_srmer, 
                df_multiIndex_basic_cooking_adoption_srmer,
               ],
    scenarios_list = [
        ['preIRA_mp8_waterHeating_adoption_srmer', 'iraRef_mp8_waterHeating_adoption_srmer'],
        ['preIRA_mp8_clothesDrying_adoption_srmer', 'iraRef_mp8_clothesDrying_adoption_srmer'],
        ['preIRA_mp8_cooking_adoption_srmer', 'iraRef_mp8_cooking_adoption_srmer']
    ],
    subplot_positions=[(0, 0), (0, 1), (0, 2),
                    #    (1, 0), (1, 1), (1, 2),
                    #    (2, 0), (2, 1), (2, 2)
                       ],
    x_labels=["", "", "",
              "", "", "",
              "", "Household Income Designation and Baseline Fuel (Pre-Retrofit)", ""
             ],
    plot_titles=["Heat Pump Water Heater:\nPre-IRA vs. IRA-Reference", "Heat Pump Clothes Dryer:\nPre-IRA vs. IRA-Reference", "Electric Resistance Range:\nPre-IRA vs. IRA-Reference"],
    y_labels=["SRMER - Retrofit Adoption Potential (%)", "", "",
             ],
    figure_size=(20,10),
    sharex=False,
    sharey=True,
#     filter_fuel=['Natural Gas', 'Electricity', 'Propane']
)

In [ ]:
df_multiIndex_basic_waterHeating_adoption_lrmer

In [ ]:
df_multiIndex_basic_waterHeating_adoption_srmer

In [ ]:
df_multiIndex_basic_clothesDrying_adoption_lrmer

In [ ]:
df_multiIndex_basic_clothesDrying_adoption_srmer

In [ ]:
df_multiIndex_basic_cooking_adoption_lrmer

In [ ]:
df_multiIndex_basic_cooking_adoption_srmer